In [30]:
import random
import os
import re
import pickle
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks
import seaborn as sns
import pickle
import torch

pd.set_option('display.expand_frame_repr', False)  # DataFrameを改行せずに表示
pd.set_option('display.max_columns', None)  # すべての列を表示

plt.rcParams["font.size"]=5
plt.rcParams["figure.figsize"]=(2.0, 1.0)
plt.rcParams["figure.dpi"]= 300

In [31]:
def extract_finalQP(filename):
    match = re.search(r'2ndQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None


def extract_1stQP(filename):
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None

def is_double_compressed(mean_difference, final_QP, threshold):    
    mean_difference = mean_difference[0]
    final_QP = final_QP[0]

    # energy_clamp = torch.clamp(mean_difference, min=0)
    # energy_clamp = torch.clamp(mean_difference)
    # energy = torch.sum(torch.square(energy_clamp))
    # mean_difference_right_clamp = torch.clamp(mean_difference[final_QP+1:52], min=0)
    energy = torch.sum(torch.square(mean_difference))
    # mean_difference_right_clamp = torch.clamp(mean_difference[final_QP+1:52])
    right_energy = torch.sum(torch.square(mean_difference[final_QP+1:52]))
        
    if energy > 0:
        energy_ratio = right_energy / energy
        if energy_ratio > threshold:
            return True
        elif energy_ratio <= threshold:
            return False
    else:
        return -1
    
def calculate_mae(file_path):
    try:
        with open(file_path, 'rb') as file:
            loaded_data, loaded_data_shifted = pickle.load(file)
    except Exception as e:
        print(f"Error occurred while loading {file_path}: {e}")
        return None

    # タプル内のリストを抽出
    original_mae = loaded_data
    shifted_mae = loaded_data_shifted

    # Coding ghostを計算してリストに格納する
    mae_difference = [shifted - original for original, shifted in zip(original_mae, shifted_mae)]
    
    # mae_differenceの各要素においてマイナスの値を0に変換
    mae_difference_positive = [0 if val < 0 else val for val in mae_difference]
    
    # mae_difference_positiveをtensorに変換
    mae_difference_tensor = torch.tensor(mae_difference_positive)
    
    
    return mae_difference_positive, mae_difference_tensor
    

In [32]:
rootpath_csv = "/Prove/Yoshihisa/HEIF_ghost/HEIF_IMAGES_CSV/"

single_path1 = os.path.join(rootpath_csv, 'HEIF_images_single_csv')
single_path2 = os.path.join(rootpath_csv, 'HEIF_images_second_sameQP_csv')
single_list1 = [os.path.join(single_path1, file) for file in sorted(os.listdir(single_path1))]
single_list2 = [os.path.join(single_path2, file) for file in sorted(os.listdir(single_path2))]

second_largeQP1_path1 = os.path.join(rootpath_csv, 'HEIF_images_second_csv')
second_largeQP1_path2 = os.path.join(rootpath_csv, 'HEIF_images_triple_csv')
second_largeQP1_list1 = [os.path.join(second_largeQP1_path1, file) for file in sorted(os.listdir(second_largeQP1_path1))]
second_largeQP1_list2 = [os.path.join(second_largeQP1_path2, file) for file in sorted(os.listdir(second_largeQP1_path2))]

second_sameQP_path1 = os.path.join(rootpath_csv, 'HEIF_images_second_sameQP_csv')
second_sameQP_path2 = os.path.join(rootpath_csv, 'HEIF_images_triple_sameQP_csv')
second_sameQP_list1 = [os.path.join(second_sameQP_path1, file) for file in sorted(os.listdir(second_sameQP_path1))]
second_sameQP_list2 = [os.path.join(second_sameQP_path2, file) for file in sorted(os.listdir(second_sameQP_path2))]

second_largeQP2_path1 = os.path.join(rootpath_csv, 'HEIF_images_second_largeQP_csv')
second_largeQP2_path2 = os.path.join(rootpath_csv, 'HEIF_images_triple_largeQP_csv')
second_largeQP2_list1 = [os.path.join(second_largeQP2_path1, file) for file in sorted(os.listdir(second_largeQP2_path1))]
second_largeQP2_list2 = [os.path.join(second_largeQP2_path2, file) for file in sorted(os.listdir(second_largeQP2_path2))]

print("single_list1: ", len(single_list1))
print("single_list2: ", len(single_list2))
print()
print("second_largeQP1_list1: ", len(second_largeQP1_list1))
print("second_largeQP1_list2: ", len(second_largeQP1_list2))
print("second_sameQP_list1: ", len(second_sameQP_list1))
print("second_sameQP_list2: ", len(second_sameQP_list2))
print("second_largeQP_list1: ", len(second_largeQP2_list1))
print("second_largeQP_list2: ", len(second_largeQP2_list2))


single_list1:  3080
single_list2:  3080

second_largeQP1_list1:  17556
second_largeQP1_list2:  17556
second_sameQP_list1:  3080
second_sameQP_list2:  3080
second_largeQP_list1:  12012
second_largeQP_list2:  12012


In [33]:
rootpath_pkl = "/Prove/Yoshihisa/HEIF_ghost/PKL/"

single_pathA = os.path.join(rootpath_pkl, 'pkl_single')
single_pathB = os.path.join(rootpath_pkl, 'pkl_second_sameQP')
single_listA = [os.path.join(single_pathA, file) for file in sorted(os.listdir(single_pathA))]
single_listB = [os.path.join(single_pathB, file) for file in sorted(os.listdir(single_pathB))]

second_largeQP1_pathA = os.path.join(rootpath_pkl, 'pkl_second')
second_largeQP1_pathB = os.path.join(rootpath_pkl, 'pkl_triple')
second_largeQP1_listA = [os.path.join(second_largeQP1_pathA, file) for file in sorted(os.listdir(second_largeQP1_pathA))]
second_largeQP1_listB = [os.path.join(second_largeQP1_pathB, file) for file in sorted(os.listdir(second_largeQP1_pathB))]

second_sameQP_pathA = os.path.join(rootpath_pkl, 'pkl_second_sameQP')
second_sameQP_pathB = os.path.join(rootpath_pkl, 'pkl_triple_sameQP')
second_sameQP_listA = [os.path.join(second_sameQP_pathA, file) for file in sorted(os.listdir(second_sameQP_pathA))]
second_sameQP_listB = [os.path.join(second_sameQP_pathB, file) for file in sorted(os.listdir(second_sameQP_pathB))]

second_largeQP2_pathA = os.path.join(rootpath_pkl, 'pkl_second_largeQP')
second_largeQP2_pathB = os.path.join(rootpath_pkl, 'pkl_triple_largeQP')
second_largeQP2_listA = [os.path.join(second_largeQP2_pathA, file) for file in sorted(os.listdir(second_largeQP2_pathA))]
second_largeQP2_listB = [os.path.join(second_largeQP2_pathB, file) for file in sorted(os.listdir(second_largeQP2_pathB))]

print("single_listA: ", len(single_listA))
print("single_listB: ", len(single_listB))
print()
print("second_largeQP1_listA: ", len(second_largeQP1_listA))
print("second_largeQP1_listB: ", len(second_largeQP1_listB))
print("second_sameQP_listA: ", len(second_sameQP_listA))
print("second_sameQP_listB: ", len(second_sameQP_listB))
print("second_largeQP2_listA: ", len(second_largeQP2_listA))
print("second_largeQP2_listB: ", len(second_largeQP2_listB))

single_listA:  3080
single_listB:  3080

second_largeQP1_listA:  17556
second_largeQP1_listB:  17556
second_sameQP_listA:  3080
second_sameQP_listB:  3080
second_largeQP2_listA:  12012
second_largeQP2_listB:  12012


In [34]:
single_csv1 = list(zip(single_list1, single_listA, single_list2, single_listB))
single_csv = random.sample(single_csv1, 2070)
print(len(single_csv))

single_test_csv = [item for item in single_csv1 if item not in single_csv]
single_test_csv1 = random.sample(single_test_csv, 207)
# single_test_csv2 = random.sample(single_test_csv, 299)
print(len(single_test_csv1))
# print(len(single_test_csv2))

2070
207


In [35]:
QP1D1 = ["_1stQP25_2ndQP24", "_1stQP40_2ndQP39"]
QP1D3 = ["_1stQP30_2ndQP27", "_1stQP35_2ndQP32", "_1stQP45_2ndQP42"]
QP1D4 = ["_1stQP20_2ndQP16"]
QP1D5 = ["_1stQP10_2ndQP5", "_1stQP15_2ndQP10", "_1stQP25_2ndQP20", "_1stQP32_2ndQP27", "_1stQP50_2ndQP45"]
QP1D6 = ["_1stQP30_2ndQP24", "_1stQP45_2ndQP39"]
QP1D8 = ["_1stQP32_2ndQP24", "_1stQP35_2ndQP27", "_1stQP40_2ndQP32", "_1stQP50_2ndQP42"]
QP1D9 = ["_1stQP25_2ndQP16"]
QP1D10 = ["_1stQP15_2ndQP5", "_1stQP20_2ndQP10", "_1stQP30_2ndQP20"]
QP1D11 = ["_1stQP35_2ndQP24", "_1stQP50_2ndQP39"]
QP1D12 = ["_1stQP32_2ndQP20"]
QP1D13 = ["_1stQP40_2ndQP27", "_1stQP45_2ndQP32"]
QP1D14 = ["_1stQP30_2ndQP16"]
QP1D15 = ["_1stQP20_2ndQP5", "_1stQP25_2ndQP10", "_1stQP35_2ndQP20"]
QP1D16 = ["_1stQP32_2ndQP16", "_1stQP40_2ndQP24"]
QP1D18 = ["_1stQP45_2ndQP27", "_1stQP50_2ndQP32"]
QP1D19 = ["_1stQP35_2ndQP16"]
QP1D20 = ["_1stQP25_2ndQP5", "_1stQP30_2ndQP10", "_1stQP40_2ndQP20"]
QP1D21 = ["_1stQP45_2ndQP24"]
QP1D22 = ["_1stQP32_2ndQP10"]
QP1D23 = ["_1stQP50_2ndQP27"]
QP1D24 = ["_1stQP40_2ndQP16"]
QP1D25 = ["_1stQP30_2ndQP5", "_1stQP35_2ndQP10", "_1stQP45_2ndQP20"]
QP1D26 = ["_1stQP50_2ndQP24"]
QP1D27 = ["_1stQP32_2ndQP5"]
QP1D29 = ["_1stQP45_2ndQP16"]
QP1D30 = ["_1stQP35_2ndQP5", "_1stQP40_2ndQP10", "_1stQP50_2ndQP20"]
QP1D34 = ["_1stQP50_2ndQP16"]
QP1D35 = ["_1stQP40_2ndQP5", "_1stQP45_2ndQP10"]
QP1D40 = ["_1stQP45_2ndQP5", "_1stQP50_2ndQP10"]
QP1D45 = ["_1stQP50_2ndQP5"]

In [36]:
second_largeQP1_csv1 = list(zip(second_largeQP1_list1, second_largeQP1_listA, second_largeQP1_list2, second_largeQP1_listB))

# 条件に合致する要素を抽出
QP1D1_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D1)]
QP1D3_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D3)]
QP1D4_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D4)]
QP1D5_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D5)]
QP1D6_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D6)]
QP1D8_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D8)]
QP1D9_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D9)]
QP1D10_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D10)]
QP1D11_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D11)]
QP1D12_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D12)]
QP1D13_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D13)]
QP1D14_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D14)]
QP1D15_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D15)]
QP1D16_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D16)]
QP1D18_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D18)]
QP1D19_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D19)]
QP1D20_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D20)]
QP1D21_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D21)]
QP1D22_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D22)]
QP1D23_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D23)]
QP1D24_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D24)]
QP1D25_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D25)]
QP1D26_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D26)]
QP1D27_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D27)]
QP1D29_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D29)]
QP1D30_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D30)]
QP1D34_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D34)]
QP1D35_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D35)]
QP1D40_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D40)]
QP1D45_second_largeQP1_csv1 = [item for item in second_largeQP1_csv1 if any(target_string in item[0] for target_string in QP1D45)]


QP1D1_random_sample = random.sample(QP1D1_second_largeQP1_csv1, 23)
QP1D3_random_sample = random.sample(QP1D3_second_largeQP1_csv1, 23)
QP1D4_random_sample = random.sample(QP1D4_second_largeQP1_csv1, 23)
QP1D5_random_sample = random.sample(QP1D5_second_largeQP1_csv1, 23)
QP1D6_random_sample = random.sample(QP1D6_second_largeQP1_csv1, 23)
QP1D8_random_sample = random.sample(QP1D8_second_largeQP1_csv1, 23)
QP1D9_random_sample = random.sample(QP1D9_second_largeQP1_csv1, 23)
QP1D10_random_sample = random.sample(QP1D10_second_largeQP1_csv1, 23)
QP1D11_random_sample = random.sample(QP1D11_second_largeQP1_csv1, 23)
QP1D12_random_sample = random.sample(QP1D12_second_largeQP1_csv1, 23)
QP1D13_random_sample = random.sample(QP1D13_second_largeQP1_csv1, 23)
QP1D14_random_sample = random.sample(QP1D14_second_largeQP1_csv1, 23)
QP1D15_random_sample = random.sample(QP1D15_second_largeQP1_csv1, 23)
QP1D16_random_sample = random.sample(QP1D16_second_largeQP1_csv1, 23)
QP1D18_random_sample = random.sample(QP1D18_second_largeQP1_csv1, 23)
QP1D19_random_sample = random.sample(QP1D19_second_largeQP1_csv1, 23)
QP1D20_random_sample = random.sample(QP1D20_second_largeQP1_csv1, 23)
QP1D21_random_sample = random.sample(QP1D21_second_largeQP1_csv1, 23)
QP1D22_random_sample = random.sample(QP1D22_second_largeQP1_csv1, 23)
QP1D23_random_sample = random.sample(QP1D23_second_largeQP1_csv1, 23)
QP1D24_random_sample = random.sample(QP1D24_second_largeQP1_csv1, 23)
QP1D25_random_sample = random.sample(QP1D25_second_largeQP1_csv1, 23)
QP1D26_random_sample = random.sample(QP1D26_second_largeQP1_csv1, 23)
QP1D27_random_sample = random.sample(QP1D27_second_largeQP1_csv1, 23)
QP1D29_random_sample = random.sample(QP1D29_second_largeQP1_csv1, 23)
QP1D30_random_sample = random.sample(QP1D30_second_largeQP1_csv1, 23)
QP1D34_random_sample = random.sample(QP1D34_second_largeQP1_csv1, 23)
QP1D35_random_sample = random.sample(QP1D35_second_largeQP1_csv1, 23)
QP1D40_random_sample = random.sample(QP1D40_second_largeQP1_csv1, 23)
QP1D45_random_sample = random.sample(QP1D45_second_largeQP1_csv1, 23)


# 各リストから23個の要素をランダムに抽出してリストに格納
second_largeQP1_csv = (
    QP1D1_random_sample + QP1D3_random_sample + QP1D4_random_sample +
    QP1D5_random_sample + QP1D6_random_sample + QP1D8_random_sample +
    QP1D9_random_sample + QP1D10_random_sample + QP1D11_random_sample +
    QP1D12_random_sample + QP1D13_random_sample + QP1D14_random_sample +
    QP1D15_random_sample + QP1D16_random_sample + QP1D18_random_sample +
    QP1D19_random_sample + QP1D20_random_sample + QP1D21_random_sample +
    QP1D22_random_sample + QP1D23_random_sample + QP1D24_random_sample +
    QP1D25_random_sample + QP1D26_random_sample + QP1D27_random_sample +
    QP1D29_random_sample + QP1D30_random_sample + QP1D34_random_sample +
    QP1D35_random_sample + QP1D40_random_sample + QP1D45_random_sample
)

# 結果の表示
print("second_largeQP1_csv:", len(second_largeQP1_csv))

# 各リストから23個の要素を除いた残りの要素を取得
QP1D1_remaining = [item for item in QP1D1_second_largeQP1_csv1 if item not in QP1D1_random_sample]
QP1D3_remaining = [item for item in QP1D3_second_largeQP1_csv1 if item not in QP1D3_random_sample]
QP1D4_remaining = [item for item in QP1D4_second_largeQP1_csv1 if item not in QP1D4_random_sample]
QP1D5_remaining = [item for item in QP1D5_second_largeQP1_csv1 if item not in QP1D5_random_sample]
QP1D6_remaining = [item for item in QP1D6_second_largeQP1_csv1 if item not in QP1D6_random_sample]
QP1D8_remaining = [item for item in QP1D8_second_largeQP1_csv1 if item not in QP1D8_random_sample]
QP1D9_remaining = [item for item in QP1D9_second_largeQP1_csv1 if item not in QP1D9_random_sample]
QP1D10_remaining = [item for item in QP1D10_second_largeQP1_csv1 if item not in QP1D10_random_sample]
QP1D11_remaining = [item for item in QP1D11_second_largeQP1_csv1 if item not in QP1D11_random_sample]
QP1D12_remaining = [item for item in QP1D12_second_largeQP1_csv1 if item not in QP1D12_random_sample]
QP1D13_remaining = [item for item in QP1D13_second_largeQP1_csv1 if item not in QP1D13_random_sample]
QP1D14_remaining = [item for item in QP1D14_second_largeQP1_csv1 if item not in QP1D14_random_sample]
QP1D15_remaining = [item for item in QP1D15_second_largeQP1_csv1 if item not in QP1D15_random_sample]
QP1D16_remaining = [item for item in QP1D16_second_largeQP1_csv1 if item not in QP1D16_random_sample]
QP1D18_remaining = [item for item in QP1D18_second_largeQP1_csv1 if item not in QP1D18_random_sample]
QP1D19_remaining = [item for item in QP1D19_second_largeQP1_csv1 if item not in QP1D19_random_sample]
QP1D20_remaining = [item for item in QP1D20_second_largeQP1_csv1 if item not in QP1D20_random_sample]
QP1D21_remaining = [item for item in QP1D21_second_largeQP1_csv1 if item not in QP1D21_random_sample]
QP1D22_remaining = [item for item in QP1D22_second_largeQP1_csv1 if item not in QP1D22_random_sample]
QP1D23_remaining = [item for item in QP1D23_second_largeQP1_csv1 if item not in QP1D23_random_sample]
QP1D24_remaining = [item for item in QP1D24_second_largeQP1_csv1 if item not in QP1D24_random_sample]
QP1D25_remaining = [item for item in QP1D25_second_largeQP1_csv1 if item not in QP1D25_random_sample]
QP1D26_remaining = [item for item in QP1D26_second_largeQP1_csv1 if item not in QP1D26_random_sample]
QP1D27_remaining = [item for item in QP1D27_second_largeQP1_csv1 if item not in QP1D27_random_sample]
QP1D29_remaining = [item for item in QP1D29_second_largeQP1_csv1 if item not in QP1D29_random_sample]
QP1D30_remaining = [item for item in QP1D30_second_largeQP1_csv1 if item not in QP1D30_random_sample]
QP1D34_remaining = [item for item in QP1D34_second_largeQP1_csv1 if item not in QP1D34_random_sample]
QP1D35_remaining = [item for item in QP1D35_second_largeQP1_csv1 if item not in QP1D35_random_sample]
QP1D40_remaining = [item for item in QP1D40_second_largeQP1_csv1 if item not in QP1D40_random_sample]
QP1D45_remaining = [item for item in QP1D45_second_largeQP1_csv1 if item not in QP1D45_random_sample]


# 残りの要素からランダムに10個の要素を抽出
QP1D1_random_remaining = random.sample(QP1D1_remaining,207)
QP1D3_random_remaining = random.sample(QP1D3_remaining,207)
QP1D4_random_remaining = random.sample(QP1D4_remaining,207)
QP1D5_random_remaining = random.sample(QP1D5_remaining,207)
QP1D6_random_remaining = random.sample(QP1D6_remaining,207)
QP1D8_random_remaining = random.sample(QP1D8_remaining,207)
QP1D9_random_remaining = random.sample(QP1D9_remaining,207)
QP1D10_random_remaining = random.sample(QP1D10_remaining,207)
QP1D11_random_remaining = random.sample(QP1D11_remaining,207)
QP1D12_random_remaining = random.sample(QP1D12_remaining,207)
QP1D13_random_remaining = random.sample(QP1D13_remaining,207)
QP1D14_random_remaining = random.sample(QP1D14_remaining,207)
QP1D15_random_remaining = random.sample(QP1D15_remaining,207)
QP1D16_random_remaining = random.sample(QP1D16_remaining,207)
QP1D18_random_remaining = random.sample(QP1D18_remaining,207)
QP1D19_random_remaining = random.sample(QP1D19_remaining,207)
QP1D20_random_remaining = random.sample(QP1D20_remaining,207)
QP1D21_random_remaining = random.sample(QP1D21_remaining,207)
QP1D22_random_remaining = random.sample(QP1D22_remaining,207)
QP1D23_random_remaining = random.sample(QP1D23_remaining,207)
QP1D24_random_remaining = random.sample(QP1D24_remaining,207)
QP1D25_random_remaining = random.sample(QP1D25_remaining,207)
QP1D26_random_remaining = random.sample(QP1D26_remaining,207)
QP1D27_random_remaining = random.sample(QP1D27_remaining,207)
QP1D29_random_remaining = random.sample(QP1D29_remaining,207)
QP1D30_random_remaining = random.sample(QP1D30_remaining,207)
QP1D34_random_remaining = random.sample(QP1D34_remaining,207)
QP1D35_random_remaining = random.sample(QP1D35_remaining,207)
QP1D40_random_remaining = random.sample(QP1D40_remaining,207)
QP1D45_random_remaining = random.sample(QP1D45_remaining,207)



# 結果の表示
# print("QP1D1_random_remaining:", len(QP1D1_random_remaining))
# print("QP1D3_random_remaining:", len(QP1D3_random_remaining))
# print("QP1D4_random_remaining:", len(QP1D4_random_remaining))
# print("QP1D5_random_remaining:", len(QP1D5_random_remaining))
# print("QP1D6_random_remaining:", len(QP1D6_random_remaining))
# print("QP1D8_random_remaining:", len(QP1D8_random_remaining))
# print("QP1D9_random_remaining:", len(QP1D9_random_remaining))
# print("QP1D10_random_remaining:", len(QP1D10_random_remaining))
# print("QP1D11_random_remaining:", len(QP1D11_random_remaining))
# print("QP1D12_random_remaining:", len(QP1D12_random_remaining))
# print("QP1D13_random_remaining:", len(QP1D13_random_remaining))
# print("QP1D14_random_remaining:", len(QP1D14_random_remaining))
# print("QP1D15_random_remaining:", len(QP1D15_random_remaining))
# print("QP1D16_random_remaining:", len(QP1D16_random_remaining))
# print("QP1D18_random_remaining:", len(QP1D18_random_remaining))
# print("QP1D19_random_remaining:", len(QP1D19_random_remaining))
# print("QP1D20_random_remaining:", len(QP1D20_random_remaining))
# print("QP1D21_random_remaining:", len(QP1D21_random_remaining))
# print("QP1D22_random_remaining:", len(QP1D22_random_remaining))
# print("QP1D23_random_remaining:", len(QP1D23_random_remaining))
# print("QP1D24_random_remaining:", len(QP1D24_random_remaining))
# print("QP1D25_random_remaining:", len(QP1D25_random_remaining))
# print("QP1D26_random_remaining:", len(QP1D26_random_remaining))
# print("QP1D27_random_remaining:", len(QP1D27_random_remaining))
# print("QP1D29_random_remaining:", len(QP1D29_random_remaining))
# print("QP1D30_random_remaining:", len(QP1D30_random_remaining))
# print("QP1D34_random_remaining:", len(QP1D34_random_remaining))
# print("QP1D35_random_remaining:", len(QP1D35_random_remaining))
# print("QP1D40_random_remaining:", len(QP1D40_random_remaining))
# print("QP1D45_random_remaining:", len(QP1D45_random_remaining))

# 結果の表示
# print("second_largeQP1_test_csv:", len(second_largeQP1_test_csv))


second_largeQP1_csv: 690


In [37]:
QPS5 = ["_1stQP5_2ndQP5"]
QPS10 = ["_1stQP10_2ndQP10"]
QPS16 = ["_1stQP16_2ndQP16"]
QPS20 = ["_1stQP20_2ndQP20"]
QPS24 = ["_1stQP24_2ndQP24"]
QPS27 = ["_1stQP27_2ndQP27"]
QPS32 = ["_1stQP32_2ndQP32"]
QPS39 = ["_1stQP39_2ndQP39"]
QPS42 = ["_1stQP42_2ndQP42"]
QPS45 = ["_1stQP45_2ndQP45"]

In [38]:
second_sameQP_csv1 = list(zip(second_sameQP_list1, second_sameQP_listA, second_sameQP_list2, second_sameQP_listB))

# 条件に合致する要素を抽出
QPS5_second_sameQP_csv1 = [item for item in second_sameQP_csv1 if any(target_string in item[0] for target_string in QPS5)]
QPS10_second_sameQP_csv1 = [item for item in second_sameQP_csv1 if any(target_string in item[0] for target_string in QPS10)]
QPS16_second_sameQP_csv1 = [item for item in second_sameQP_csv1 if any(target_string in item[0] for target_string in QPS16)]
QPS20_second_sameQP_csv1 = [item for item in second_sameQP_csv1 if any(target_string in item[0] for target_string in QPS20)]
QPS24_second_sameQP_csv1 = [item for item in second_sameQP_csv1 if any(target_string in item[0] for target_string in QPS24)]
QPS27_second_sameQP_csv1 = [item for item in second_sameQP_csv1 if any(target_string in item[0] for target_string in QPS27)]
QPS32_second_sameQP_csv1 = [item for item in second_sameQP_csv1 if any(target_string in item[0] for target_string in QPS32)]
QPS39_second_sameQP_csv1 = [item for item in second_sameQP_csv1 if any(target_string in item[0] for target_string in QPS39)]
QPS42_second_sameQP_csv1 = [item for item in second_sameQP_csv1 if any(target_string in item[0] for target_string in QPS42)]
QPS45_second_sameQP_csv1 = [item for item in second_sameQP_csv1 if any(target_string in item[0] for target_string in QPS45)]

QPS5_random_sample = random.sample(QPS5_second_sameQP_csv1, 69)
QPS10_random_sample = random.sample(QPS10_second_sameQP_csv1, 69)
QPS16_random_sample = random.sample(QPS16_second_sameQP_csv1, 69)
QPS20_random_sample = random.sample(QPS20_second_sameQP_csv1, 69)
QPS24_random_sample = random.sample(QPS24_second_sameQP_csv1, 69)
QPS27_random_sample = random.sample(QPS27_second_sameQP_csv1, 69)
QPS32_random_sample = random.sample(QPS32_second_sameQP_csv1, 69)
QPS39_random_sample = random.sample(QPS39_second_sameQP_csv1, 69)
QPS42_random_sample = random.sample(QPS42_second_sameQP_csv1, 69)
QPS45_random_sample = random.sample(QPS45_second_sameQP_csv1, 69)

second_sameQP_csv = (
    QPS5_random_sample + QPS10_random_sample + QPS16_random_sample + 
    QPS20_random_sample + QPS24_random_sample + QPS27_random_sample +
    QPS32_random_sample + QPS39_random_sample + QPS42_random_sample + 
    QPS45_random_sample
)


# 結果の表示
print("second_sameQP_csv:", len(second_sameQP_csv))

# 各リストから23個の要素を除いた残りの要素を取得

QPS5_remaining = [item for item in QPS5_second_sameQP_csv1 if item not in QPS5_random_sample]
QPS10_remaining = [item for item in QPS10_second_sameQP_csv1 if item not in QPS10_random_sample]
QPS16_remaining = [item for item in QPS16_second_sameQP_csv1 if item not in QPS16_random_sample]
QPS20_remaining = [item for item in QPS20_second_sameQP_csv1 if item not in QPS20_random_sample]
QPS24_remaining = [item for item in QPS24_second_sameQP_csv1 if item not in QPS24_random_sample]
QPS27_remaining = [item for item in QPS27_second_sameQP_csv1 if item not in QPS27_random_sample]
QPS32_remaining = [item for item in QPS32_second_sameQP_csv1 if item not in QPS32_random_sample]
QPS39_remaining = [item for item in QPS39_second_sameQP_csv1 if item not in QPS39_random_sample]
QPS42_remaining = [item for item in QPS42_second_sameQP_csv1 if item not in QPS42_random_sample]
QPS45_remaining = [item for item in QPS45_second_sameQP_csv1 if item not in QPS45_random_sample]


# 残りの要素からランダムに10個の要素を抽出
QPS5_random_remaining = random.sample(QPS5_remaining, 5)
QPS10_random_remaining = random.sample(QPS10_remaining, 5)
QPS16_random_remaining = random.sample(QPS16_remaining, 5)
QPS20_random_remaining = random.sample(QPS20_remaining, 5)
QPS24_random_remaining = random.sample(QPS24_remaining, 5)
QPS27_random_remaining = random.sample(QPS27_remaining, 5)
QPS32_random_remaining = random.sample(QPS32_remaining, 5)
QPS39_random_remaining = random.sample(QPS39_remaining, 5)
QPS42_random_remaining = random.sample(QPS42_remaining, 5)
QPS45_random_remaining = random.sample(QPS45_remaining, 5)



# 結果の表示
# print("QPS5_random_remaining:", len(QPS5_random_remaining))
# print("QPS10_random_remaining:", len(QPS10_random_remaining))
# print("QPS16_random_remaining:", len(QPS16_random_remaining))
# print("QPS20_random_remaining:", len(QPS20_random_remaining))
# print("QPS24_random_remaining:", len(QPS24_random_remaining))
# print("QPS27_random_remaining:", len(QPS27_random_remaining))
# print("QPS32_random_remaining:", len(QPS32_random_remaining))
# print("QPS39_random_remaining:", len(QPS39_random_remaining))
# print("QPS42_random_remaining:", len(QPS42_random_remaining))
# print("QPS45_random_remaining:", len(QPS45_random_remaining))

# print("second_sameQP_test_csv:", len(second_sameQP_test_csv))

second_sameQP_csv: 690


In [39]:
QP2D1 = ["_1stQP15_2ndQP16"]
QP2D2 = ["_1stQP25_2ndQP27", "_1stQP30_2ndQP32", "_1stQP40_2ndQP42"]
QP2D4 = ["_1stQP20_2ndQP24", "_1stQP35_2ndQP39"]
QP2D5 = ["_1stQP15_2ndQP20", "_1stQP40_2ndQP45"]
QP2D6 = ["_1stQP10_2ndQP16"]
QP2D7 = ["_1stQP20_2ndQP27", "_1stQP25_2ndQP32", "_1stQP32_2ndQP39", "_1stQP35_2ndQP42"]
QP2D9 = ["_1stQP15_2ndQP24", "_1stQP30_2ndQP39"]
QP2D10 = ["_1stQP10_2ndQP20", "_1stQP32_2ndQP42", "_1stQP35_2ndQP45"]
QP2D12 = ["_1stQP15_2ndQP27", "_1stQP20_2ndQP32", "_1stQP30_2ndQP42"]
QP2D13 = ["_1stQP32_2ndQP45"]
QP2D14 = ["_1stQP10_2ndQP24", "_1stQP25_2ndQP39"]
QP2D15 = ["_1stQP30_2ndQP45"]
QP2D17 = ["_1stQP10_2ndQP27", "_1stQP15_2ndQP32", "_1stQP25_2ndQP42"]
QP2D19 = ["_1stQP20_2ndQP39"]
QP2D20 = ["_1stQP25_2ndQP45"]
QP2D22 = ["_1stQP10_2ndQP32", "_1stQP20_2ndQP42"]
QP2D24 = ["_1stQP15_2ndQP39"]
QP2D25 = ["_1stQP20_2ndQP45"]
QP2D27 = ["_1stQP15_2ndQP42"]
QP2D29 = ["_1stQP10_2ndQP39"]
QP2D30 = ["_1stQP15_2ndQP45"]
QP2D32 = ["_1stQP10_2ndQP42"]
QP2D35 = ["_1stQP10_2ndQP45"]

In [40]:
second_largeQP2_csv1 = list(zip(second_largeQP2_list1, second_largeQP2_listA, second_largeQP2_list2, second_largeQP2_listB))

# 条件に合致する要素を抽出
QP2D1_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D1)]
QP2D2_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D2)]
QP2D4_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D4)]
QP2D5_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D5)]
QP2D6_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D6)]
QP2D7_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D7)]
QP2D9_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D9)]
QP2D10_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D10)]
QP2D12_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D12)]
QP2D13_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D13)]
QP2D14_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D14)]
QP2D15_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D15)]
QP2D17_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D17)]
QP2D19_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D19)]
QP2D20_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D20)]
QP2D22_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D22)]
QP2D24_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D24)]
QP2D25_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D25)]
QP2D27_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D27)]
QP2D29_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D29)]
QP2D30_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D30)]
QP2D32_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D32)]
QP2D35_second_largeQP2_csv1 = [item for item in second_largeQP2_csv1 if any(target_string in item[0] for target_string in QP2D35)]

QP2D1_random_sample = random.sample(QP2D1_second_largeQP2_csv1, 30)
QP2D2_random_sample = random.sample(QP2D2_second_largeQP2_csv1, 30)
QP2D4_random_sample = random.sample(QP2D4_second_largeQP2_csv1, 30)
QP2D5_random_sample = random.sample(QP2D5_second_largeQP2_csv1, 30)
QP2D6_random_sample = random.sample(QP2D6_second_largeQP2_csv1, 30)
QP2D7_random_sample = random.sample(QP2D7_second_largeQP2_csv1, 30)
QP2D9_random_sample = random.sample(QP2D9_second_largeQP2_csv1, 30)
QP2D10_random_sample = random.sample(QP2D10_second_largeQP2_csv1, 30)
QP2D12_random_sample = random.sample(QP2D12_second_largeQP2_csv1, 30)
QP2D13_random_sample = random.sample(QP2D13_second_largeQP2_csv1, 30)
QP2D14_random_sample = random.sample(QP2D14_second_largeQP2_csv1, 30)
QP2D15_random_sample = random.sample(QP2D15_second_largeQP2_csv1, 30)
QP2D17_random_sample = random.sample(QP2D17_second_largeQP2_csv1, 30)
QP2D19_random_sample = random.sample(QP2D19_second_largeQP2_csv1, 30)
QP2D20_random_sample = random.sample(QP2D20_second_largeQP2_csv1, 30)
QP2D22_random_sample = random.sample(QP2D22_second_largeQP2_csv1, 30)
QP2D24_random_sample = random.sample(QP2D24_second_largeQP2_csv1, 30)
QP2D25_random_sample = random.sample(QP2D25_second_largeQP2_csv1, 30)
QP2D27_random_sample = random.sample(QP2D27_second_largeQP2_csv1, 30)
QP2D29_random_sample = random.sample(QP2D29_second_largeQP2_csv1, 30)
QP2D30_random_sample = random.sample(QP2D30_second_largeQP2_csv1, 30)
QP2D32_random_sample = random.sample(QP2D32_second_largeQP2_csv1, 30)
QP2D35_random_sample = random.sample(QP2D35_second_largeQP2_csv1, 30)


# 各リストから23個の要素をランダムに抽出してリストに格納
second_largeQP2_csv = (
    QP2D1_random_sample + QP2D2_random_sample + QP2D4_random_sample +
    QP2D5_random_sample + QP2D6_random_sample + QP2D7_random_sample +
    QP2D9_random_sample + QP2D10_random_sample + QP2D12_random_sample + 
    QP2D13_random_sample + QP2D14_random_sample + QP2D15_random_sample + 
    QP2D17_random_sample + QP2D19_random_sample + QP2D20_random_sample +
    QP2D22_random_sample + QP2D24_random_sample + QP2D25_random_sample + 
    QP2D27_random_sample + QP2D29_random_sample + QP2D30_random_sample + 
    QP2D32_random_sample + QP2D35_random_sample)

# 結果の表示
print("second_largeQP2_csv:", len(second_largeQP2_csv))

# 各リストから23個の要素を除いた残りの要素を取得
QP2D1_remaining = [item for item in QP2D1_second_largeQP2_csv1 if item not in QP2D1_random_sample]
QP2D2_remaining = [item for item in QP2D2_second_largeQP2_csv1 if item not in QP2D2_random_sample]
QP2D4_remaining = [item for item in QP2D4_second_largeQP2_csv1 if item not in QP2D4_random_sample]
QP2D5_remaining = [item for item in QP2D5_second_largeQP2_csv1 if item not in QP2D5_random_sample]
QP2D6_remaining = [item for item in QP2D6_second_largeQP2_csv1 if item not in QP2D6_random_sample]
QP2D7_remaining = [item for item in QP2D7_second_largeQP2_csv1 if item not in QP2D7_random_sample]
QP2D9_remaining = [item for item in QP2D9_second_largeQP2_csv1 if item not in QP2D9_random_sample]
QP2D10_remaining = [item for item in QP2D10_second_largeQP2_csv1 if item not in QP2D10_random_sample]
QP2D12_remaining = [item for item in QP2D12_second_largeQP2_csv1 if item not in QP2D12_random_sample]
QP2D13_remaining = [item for item in QP2D13_second_largeQP2_csv1 if item not in QP2D13_random_sample]
QP2D14_remaining = [item for item in QP2D14_second_largeQP2_csv1 if item not in QP2D14_random_sample]
QP2D15_remaining = [item for item in QP2D15_second_largeQP2_csv1 if item not in QP2D15_random_sample]
QP2D17_remaining = [item for item in QP2D17_second_largeQP2_csv1 if item not in QP2D17_random_sample]
QP2D19_remaining = [item for item in QP2D19_second_largeQP2_csv1 if item not in QP2D19_random_sample]
QP2D20_remaining = [item for item in QP2D20_second_largeQP2_csv1 if item not in QP2D20_random_sample]
QP2D22_remaining = [item for item in QP2D22_second_largeQP2_csv1 if item not in QP2D22_random_sample]
QP2D24_remaining = [item for item in QP2D24_second_largeQP2_csv1 if item not in QP2D24_random_sample]
QP2D25_remaining = [item for item in QP2D25_second_largeQP2_csv1 if item not in QP2D25_random_sample]
QP2D27_remaining = [item for item in QP2D27_second_largeQP2_csv1 if item not in QP2D27_random_sample]
QP2D29_remaining = [item for item in QP2D29_second_largeQP2_csv1 if item not in QP2D29_random_sample]
QP2D30_remaining = [item for item in QP2D30_second_largeQP2_csv1 if item not in QP2D30_random_sample]
QP2D32_remaining = [item for item in QP2D32_second_largeQP2_csv1 if item not in QP2D32_random_sample]
QP2D35_remaining = [item for item in QP2D35_second_largeQP2_csv1 if item not in QP2D35_random_sample]



# 残りの要素からランダムに10個の要素を抽出
QP2D1_random_remaining = random.sample(QP2D1_remaining,207)
QP2D2_random_remaining = random.sample(QP2D2_remaining,207)
QP2D4_random_remaining = random.sample(QP2D4_remaining,207)
QP2D5_random_remaining = random.sample(QP2D5_remaining,207)
QP2D6_random_remaining = random.sample(QP2D6_remaining,207)
QP2D7_random_remaining = random.sample(QP2D7_remaining,207)
QP2D9_random_remaining = random.sample(QP2D9_remaining,207)
QP2D10_random_remaining = random.sample(QP2D10_remaining,207)
QP2D12_random_remaining = random.sample(QP2D12_remaining,207)
QP2D13_random_remaining = random.sample(QP2D13_remaining,207)
QP2D14_random_remaining = random.sample(QP2D14_remaining,207)
QP2D15_random_remaining = random.sample(QP2D15_remaining,207)
QP2D17_random_remaining = random.sample(QP2D17_remaining,207)
QP2D19_random_remaining = random.sample(QP2D19_remaining,207)
QP2D20_random_remaining = random.sample(QP2D20_remaining,207)
QP2D22_random_remaining = random.sample(QP2D22_remaining,207)
QP2D24_random_remaining = random.sample(QP2D24_remaining,207)
QP2D25_random_remaining = random.sample(QP2D25_remaining,207)
QP2D27_random_remaining = random.sample(QP2D27_remaining,207)
QP2D29_random_remaining = random.sample(QP2D29_remaining,207)
QP2D30_random_remaining = random.sample(QP2D30_remaining,207)
QP2D32_random_remaining = random.sample(QP2D32_remaining,207)
QP2D35_random_remaining = random.sample(QP2D35_remaining,207)


# 結果の表示

# print("QP2D1_random_remaining:", len(QP2D1_random_remaining))
# print("QP2D2_random_remaining:", len(QP2D2_random_remaining))
# print("QP2D4_random_remaining:", len(QP2D4_random_remaining))
# print("QP2D5_random_remaining:", len(QP2D5_random_remaining))
# print("QP2D6_random_remaining:", len(QP2D6_random_remaining))
# print("QP2D7_random_remaining:", len(QP2D7_random_remaining))
# print("QP2D9_random_remaining:", len(QP2D9_random_remaining))
# print("QP2D10_random_remaining:", len(QP2D10_random_remaining))
# print("QP2D12_random_remaining:", len(QP2D12_random_remaining))
# print("QP2D13_random_remaining:", len(QP2D13_random_remaining))
# print("QP2D14_random_remaining:", len(QP2D14_random_remaining))
# print("QP2D15_random_remaining:", len(QP2D15_random_remaining))
# print("QP2D17_random_remaining:", len(QP2D17_random_remaining))
# print("QP2D19_random_remaining:", len(QP2D19_random_remaining))
# print("QP2D20_random_remaining:", len(QP2D20_random_remaining))
# print("QP2D22_random_remaining:", len(QP2D22_random_remaining))
# print("QP2D24_random_remaining:", len(QP2D24_random_remaining))
# print("QP2D25_random_remaining:", len(QP2D25_random_remaining))
# print("QP2D27_random_remaining:", len(QP2D27_random_remaining))
# print("QP2D29_random_remaining:", len(QP2D29_random_remaining))
# print("QP2D30_random_remaining:", len(QP2D30_random_remaining))
# print("QP2D32_random_remaining:", len(QP2D32_random_remaining))
# print("QP2D35_random_remaining:", len(QP2D35_random_remaining))


# print("second_largeQP2_test_csv:", len(second_largeQP2_test_csv))

second_largeQP2_csv: 690


In [41]:
train_csv_list = single_csv + second_largeQP1_csv + second_sameQP_csv + second_largeQP2_csv
# train_csv_list = single_csv + second_largeQP1_csv
print("train_csv_list: ", len(train_csv_list))

# largeQP1
test_csv_QP1D1_list = single_test_csv1 + QP1D1_random_remaining
test_csv_QP1D3_list = single_test_csv1 + QP1D3_random_remaining
test_csv_QP1D4_list = single_test_csv1 + QP1D4_random_remaining
test_csv_QP1D5_list = single_test_csv1 + QP1D5_random_remaining
test_csv_QP1D6_list = single_test_csv1 + QP1D6_random_remaining
test_csv_QP1D8_list = single_test_csv1 + QP1D8_random_remaining
test_csv_QP1D9_list = single_test_csv1 + QP1D9_random_remaining
test_csv_QP1D10_list = single_test_csv1 + QP1D10_random_remaining
test_csv_QP1D11_list = single_test_csv1 + QP1D11_random_remaining
test_csv_QP1D12_list = single_test_csv1 + QP1D12_random_remaining
test_csv_QP1D13_list = single_test_csv1 + QP1D13_random_remaining
test_csv_QP1D14_list = single_test_csv1 + QP1D14_random_remaining
test_csv_QP1D15_list = single_test_csv1 + QP1D15_random_remaining
test_csv_QP1D16_list = single_test_csv1 + QP1D16_random_remaining
test_csv_QP1D18_list = single_test_csv1 + QP1D18_random_remaining
test_csv_QP1D19_list = single_test_csv1 + QP1D19_random_remaining
test_csv_QP1D20_list = single_test_csv1 + QP1D20_random_remaining
test_csv_QP1D21_list = single_test_csv1 + QP1D21_random_remaining
test_csv_QP1D22_list = single_test_csv1 + QP1D22_random_remaining
test_csv_QP1D23_list = single_test_csv1 + QP1D23_random_remaining
test_csv_QP1D24_list = single_test_csv1 + QP1D24_random_remaining
test_csv_QP1D25_list = single_test_csv1 + QP1D25_random_remaining
test_csv_QP1D26_list = single_test_csv1 + QP1D26_random_remaining
test_csv_QP1D27_list = single_test_csv1 + QP1D27_random_remaining
test_csv_QP1D29_list = single_test_csv1 + QP1D29_random_remaining
test_csv_QP1D30_list = single_test_csv1 + QP1D30_random_remaining
test_csv_QP1D34_list = single_test_csv1 + QP1D34_random_remaining
test_csv_QP1D35_list = single_test_csv1 + QP1D35_random_remaining
test_csv_QP1D40_list = single_test_csv1 + QP1D40_random_remaining
test_csv_QP1D45_list = single_test_csv1 + QP1D45_random_remaining
# sameQP
test_csv_QPS5_list = single_test_csv1 + QPS5_random_remaining
test_csv_QPS10_list = single_test_csv1 + QPS10_random_remaining
test_csv_QPS16_list = single_test_csv1 + QPS16_random_remaining
test_csv_QPS20_list = single_test_csv1 + QPS20_random_remaining
test_csv_QPS24_list = single_test_csv1 + QPS24_random_remaining
test_csv_QPS27_list = single_test_csv1 + QPS27_random_remaining
test_csv_QPS32_list = single_test_csv1 + QPS32_random_remaining
test_csv_QPS39_list = single_test_csv1 + QPS39_random_remaining
test_csv_QPS42_list = single_test_csv1 + QPS42_random_remaining
test_csv_QPS45_list = single_test_csv1 + QPS45_random_remaining
# largeQP2
test_csv_QP2D1_list = single_test_csv1 + QP2D1_random_remaining
test_csv_QP2D2_list = single_test_csv1 + QP2D2_random_remaining
test_csv_QP2D4_list = single_test_csv1 + QP2D4_random_remaining
test_csv_QP2D5_list = single_test_csv1 + QP2D5_random_remaining
test_csv_QP2D6_list = single_test_csv1 + QP2D6_random_remaining
test_csv_QP2D7_list = single_test_csv1 + QP2D7_random_remaining
test_csv_QP2D9_list = single_test_csv1 + QP2D9_random_remaining
test_csv_QP2D10_list = single_test_csv1 + QP2D10_random_remaining
test_csv_QP2D12_list = single_test_csv1 + QP2D12_random_remaining
test_csv_QP2D13_list = single_test_csv1 + QP2D13_random_remaining
test_csv_QP2D14_list = single_test_csv1 + QP2D14_random_remaining
test_csv_QP2D15_list = single_test_csv1 + QP2D15_random_remaining
test_csv_QP2D17_list = single_test_csv1 + QP2D17_random_remaining
test_csv_QP2D19_list = single_test_csv1 + QP2D19_random_remaining
test_csv_QP2D20_list = single_test_csv1 + QP2D20_random_remaining
test_csv_QP2D22_list = single_test_csv1 + QP2D22_random_remaining
test_csv_QP2D24_list = single_test_csv1 + QP2D24_random_remaining
test_csv_QP2D25_list = single_test_csv1 + QP2D25_random_remaining
test_csv_QP2D27_list = single_test_csv1 + QP2D27_random_remaining
test_csv_QP2D29_list = single_test_csv1 + QP2D29_random_remaining
test_csv_QP2D30_list = single_test_csv1 + QP2D30_random_remaining
test_csv_QP2D32_list = single_test_csv1 + QP2D32_random_remaining
test_csv_QP2D35_list = single_test_csv1 + QP2D35_random_remaining

print("test_csv_QP1D1_data:", len(test_csv_QP1D1_list))
print("test_csv_QP1D3_data:", len(test_csv_QP1D3_list))
print("test_csv_QP1D4_data:", len(test_csv_QP1D4_list))
print("test_csv_QP1D5_data:", len(test_csv_QP1D5_list))
print("test_csv_QP1D6_data:", len(test_csv_QP1D6_list))
print("test_csv_QP1D8_data:", len(test_csv_QP1D8_list))
print("test_csv_QP1D9_data:", len(test_csv_QP1D9_list))
print("test_csv_QP1D10_data:", len(test_csv_QP1D10_list))
print("test_csv_QP1D11_data:", len(test_csv_QP1D11_list))
print("test_csv_QP1D12_data:", len(test_csv_QP1D12_list))
print("test_csv_QP1D13_data:", len(test_csv_QP1D13_list))
print("test_csv_QP1D14_data:", len(test_csv_QP1D14_list))
print("test_csv_QP1D15_data:", len(test_csv_QP1D15_list))
print("test_csv_QP1D16_data:", len(test_csv_QP1D16_list))
print("test_csv_QP1D18_data:", len(test_csv_QP1D18_list))
print("test_csv_QP1D19_data:", len(test_csv_QP1D19_list))
print("test_csv_QP1D20_data:", len(test_csv_QP1D20_list))
print("test_csv_QP1D21_data:", len(test_csv_QP1D21_list))
print("test_csv_QP1D22_data:", len(test_csv_QP1D22_list))
print("test_csv_QP1D23_data:", len(test_csv_QP1D23_list))
print("test_csv_QP1D24_data:", len(test_csv_QP1D24_list))
print("test_csv_QP1D25_data:", len(test_csv_QP1D25_list))
print("test_csv_QP1D26_data:", len(test_csv_QP1D26_list))
print("test_csv_QP1D27_data:", len(test_csv_QP1D27_list))
print("test_csv_QP1D29_data:", len(test_csv_QP1D29_list))
print("test_csv_QP1D30_data:", len(test_csv_QP1D30_list))
print("test_csv_QP1D34_data:", len(test_csv_QP1D34_list))
print("test_csv_QP1D35_data:", len(test_csv_QP1D35_list))
print("test_csv_QP1D40_data:", len(test_csv_QP1D40_list))
print("test_csv_QP1D45_data:", len(test_csv_QP1D45_list))
print()
print("test_csv_QPS5_data:", len(test_csv_QPS5_list))
print("test_csv_QPS10_data:", len(test_csv_QPS10_list))
print("test_csv_QPS16_data:", len(test_csv_QPS16_list))
print("test_csv_QPS20_data:", len(test_csv_QPS20_list))
print("test_csv_QPS24_data:", len(test_csv_QPS24_list))
print("test_csv_QPS27_data:", len(test_csv_QPS27_list))
print("test_csv_QPS32_data:", len(test_csv_QPS32_list))
print("test_csv_QPS39_data:", len(test_csv_QPS39_list))
print("test_csv_QPS42_data:", len(test_csv_QPS42_list))
print("test_csv_QPS45_data:", len(test_csv_QPS45_list))
print()
print("test_csv_QP2D1_data:", len(test_csv_QP2D1_list))
print("test_csv_QP2D2_data:", len(test_csv_QP2D2_list))
print("test_csv_QP2D4_data:", len(test_csv_QP2D4_list))
print("test_csv_QP2D5_data:", len(test_csv_QP2D5_list))
print("test_csv_QP2D6_data:", len(test_csv_QP2D6_list))
print("test_csv_QP2D7_data:", len(test_csv_QP2D7_list))
print("test_csv_QP2D9_data:", len(test_csv_QP2D9_list))
print("test_csv_QP2D10_data:", len(test_csv_QP2D10_list))
print("test_csv_QP2D12_data:", len(test_csv_QP2D12_list))
print("test_csv_QP2D13_data:", len(test_csv_QP2D13_list))
print("test_csv_QP2D14_data:", len(test_csv_QP2D14_list))
print("test_csv_QP2D15_data:", len(test_csv_QP2D15_list))
print("test_csv_QP2D17_data:", len(test_csv_QP2D17_list))
print("test_csv_QP2D19_data:", len(test_csv_QP2D19_list))
print("test_csv_QP2D20_data:", len(test_csv_QP2D20_list))
print("test_csv_QP2D22_data:", len(test_csv_QP2D22_list))
print("test_csv_QP2D24_data:", len(test_csv_QP2D24_list))
print("test_csv_QP2D25_data:", len(test_csv_QP2D25_list))
print("test_csv_QP2D27_data:", len(test_csv_QP2D27_list))
print("test_csv_QP2D29_data:", len(test_csv_QP2D29_list))
print("test_csv_QP2D30_data:", len(test_csv_QP2D30_list))
print("test_csv_QP2D32_data:", len(test_csv_QP2D32_list))
print("test_csv_QP2D35_data:", len(test_csv_QP2D35_list))


train_csv_list:  4140
test_csv_QP1D1_data: 414
test_csv_QP1D3_data: 414
test_csv_QP1D4_data: 414
test_csv_QP1D5_data: 414
test_csv_QP1D6_data: 414
test_csv_QP1D8_data: 414
test_csv_QP1D9_data: 414
test_csv_QP1D10_data: 414
test_csv_QP1D11_data: 414
test_csv_QP1D12_data: 414
test_csv_QP1D13_data: 414
test_csv_QP1D14_data: 414
test_csv_QP1D15_data: 414
test_csv_QP1D16_data: 414
test_csv_QP1D18_data: 414
test_csv_QP1D19_data: 414
test_csv_QP1D20_data: 414
test_csv_QP1D21_data: 414
test_csv_QP1D22_data: 414
test_csv_QP1D23_data: 414
test_csv_QP1D24_data: 414
test_csv_QP1D25_data: 414
test_csv_QP1D26_data: 414
test_csv_QP1D27_data: 414
test_csv_QP1D29_data: 414
test_csv_QP1D30_data: 414
test_csv_QP1D34_data: 414
test_csv_QP1D35_data: 414
test_csv_QP1D40_data: 414
test_csv_QP1D45_data: 414

test_csv_QPS5_data: 212
test_csv_QPS10_data: 212
test_csv_QPS16_data: 212
test_csv_QPS20_data: 212
test_csv_QPS24_data: 212
test_csv_QPS27_data: 212
test_csv_QPS32_data: 212
test_csv_QPS39_data: 212
test_

In [42]:
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
train_df1_1 = pd.DataFrame(columns=pu_columns)
train_df1_2 = pd.DataFrame(columns=luminance_columns)
train_df1_3 = pd.DataFrame(columns=chrominance_columns)
train_df2 = pd.DataFrame(columns=label_columns)
train_df3 = pd.DataFrame(columns=mae1_columns)
train_df4 = pd.DataFrame(columns=mae2_columns)
train_df5 = pd.DataFrame(columns=mae_columns)
train_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in train_csv_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    train_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]
    
    train_df1_1 = pd.concat([train_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    train_df1_2= pd.concat([train_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    train_df1_3 = pd.concat([train_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)

    # label_columnsの値を取得
    train_df2 = pd.concat([train_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    final_QP = extract_finalQP(train_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(train_pkl_list[0])
    mae_d2, _ = calculate_mae(train_pkl_list[1])
    
    # mae1_columnsの値を取得
    train_df3 = pd.concat([train_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    train_df4 = pd.concat([train_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)

    # mae_columnsの値を取得
    train_df5 = pd.concat([train_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    train_df6 = pd.concat([train_df6, pd.DataFrame({"FINAL_QP": [final_QP]})], ignore_index=True)

# インデックスをリセット
train_df1_1.reset_index(drop=True, inplace=True)
train_df1_2.reset_index(drop=True, inplace=True)
train_df1_3.reset_index(drop=True, inplace=True)
train_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
train_df = pd.concat([train_df1_1, train_df1_2, train_df1_3, train_df3, train_df4], axis=1)
train_df_onlyGhost = pd.concat([train_df3, train_df4], axis=1)

In [14]:
# スケーラーを使って結合したデータをスケーリング
X_train = scaler.fit_transform(train_df)
X_train_onlyGhost = scaler.fit_transform(train_df_onlyGhost)

# pandasをndarrayに変換
MAE = train_df5.values
FINAL_QP = train_df6.values

# ラベルの準備
Y_train = train_df2['LABEL'].astype(int)

print(f'Length of X_train: {len(X_train)}')
print(f'Length of X_train_onlyGhost: {len(X_train_onlyGhost)}')
print(f'Length of Y_train: {len(Y_train)}')
print(f'Length of MAE: {len(MAE)}')
print(f'Length of FINAL_QP: {len(FINAL_QP)}')

Length of X_train: 5520
Length of X_train_onlyGhost: 5520
Length of Y_train: 5520
Length of MAE: 5520
Length of FINAL_QP: 5520


In [15]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D1_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D1_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D1_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D1_df2 = pd.DataFrame(columns=label_columns)
test_QP1D1_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D1_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D1_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D1_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D1_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D1_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D1_df1_1 = pd.concat([test_QP1D1_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D1_df1_2 = pd.concat([test_QP1D1_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D1_df1_3 = pd.concat([test_QP1D1_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D1_df2 = pd.concat([test_QP1D1_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D1_final_QP = extract_finalQP(test_QP1D1_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D1_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D1_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D1_df3 = pd.concat([test_QP1D1_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D1_df4 = pd.concat([test_QP1D1_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D1_df5 = pd.concat([test_QP1D1_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D1_df6 = pd.concat([test_QP1D1_df6, pd.DataFrame({"FINAL_QP": [test_QP1D1_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D1_df1_1.reset_index(drop=True, inplace=True)
test_QP1D1_df1_2.reset_index(drop=True, inplace=True)
test_QP1D1_df1_3.reset_index(drop=True, inplace=True)
test_QP1D1_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D1 = pd.concat([test_QP1D1_df1_1, test_QP1D1_df1_2, test_QP1D1_df1_3, test_QP1D1_df3, test_QP1D1_df4], axis=1)
test_QP1D1_onlyGhost = pd.concat([test_QP1D1_df3, test_QP1D1_df4], axis=1)


In [16]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D3_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D3_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D3_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D3_df2 = pd.DataFrame(columns=label_columns)
test_QP1D3_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D3_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D3_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D3_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D3_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D3_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D3_df1_1 = pd.concat([test_QP1D3_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D3_df1_2 = pd.concat([test_QP1D3_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D3_df1_3 = pd.concat([test_QP1D3_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D3_df2 = pd.concat([test_QP1D3_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D3_final_QP = extract_finalQP(test_QP1D3_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D3_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D3_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D3_df3 = pd.concat([test_QP1D3_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D3_df4 = pd.concat([test_QP1D3_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D3_df5 = pd.concat([test_QP1D3_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D3_df6 = pd.concat([test_QP1D3_df6, pd.DataFrame({"FINAL_QP": [test_QP1D3_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D3_df1_1.reset_index(drop=True, inplace=True)
test_QP1D3_df1_2.reset_index(drop=True, inplace=True)
test_QP1D3_df1_3.reset_index(drop=True, inplace=True)
test_QP1D3_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D3 = pd.concat([test_QP1D3_df1_1, test_QP1D3_df1_2, test_QP1D3_df1_3, test_QP1D3_df3, test_QP1D3_df4], axis=1)
test_QP1D3_onlyGhost = pd.concat([test_QP1D3_df3, test_QP1D3_df4], axis=1)


In [17]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D4_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D4_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D4_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D4_df2 = pd.DataFrame(columns=label_columns)
test_QP1D4_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D4_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D4_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D4_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D4_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D4_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D4_df1_1 = pd.concat([test_QP1D4_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D4_df1_2 = pd.concat([test_QP1D4_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D4_df1_3 = pd.concat([test_QP1D4_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D4_df2 = pd.concat([test_QP1D4_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D4_final_QP = extract_finalQP(test_QP1D4_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D4_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D4_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D4_df3 = pd.concat([test_QP1D4_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D4_df4 = pd.concat([test_QP1D4_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D4_df5 = pd.concat([test_QP1D4_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D4_df6 = pd.concat([test_QP1D4_df6, pd.DataFrame({"FINAL_QP": [test_QP1D4_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D4_df1_1.reset_index(drop=True, inplace=True)
test_QP1D4_df1_2.reset_index(drop=True, inplace=True)
test_QP1D4_df1_3.reset_index(drop=True, inplace=True)
test_QP1D4_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D4 = pd.concat([test_QP1D4_df1_1, test_QP1D4_df1_2, test_QP1D4_df1_3, test_QP1D4_df3, test_QP1D4_df4], axis=1)
test_QP1D4_onlyGhost = pd.concat([test_QP1D4_df3, test_QP1D4_df4], axis=1)


In [18]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D5_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D5_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D5_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D5_df2 = pd.DataFrame(columns=label_columns)
test_QP1D5_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D5_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D5_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D5_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D5_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D5_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D5_df1_1 = pd.concat([test_QP1D5_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D5_df1_2 = pd.concat([test_QP1D5_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D5_df1_3 = pd.concat([test_QP1D5_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D5_df2 = pd.concat([test_QP1D5_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D5_final_QP = extract_finalQP(test_QP1D5_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D5_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D5_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D5_df3 = pd.concat([test_QP1D5_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D5_df4 = pd.concat([test_QP1D5_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D5_df5 = pd.concat([test_QP1D5_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D5_df6 = pd.concat([test_QP1D5_df6, pd.DataFrame({"FINAL_QP": [test_QP1D5_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D5_df1_1.reset_index(drop=True, inplace=True)
test_QP1D5_df1_2.reset_index(drop=True, inplace=True)
test_QP1D5_df1_3.reset_index(drop=True, inplace=True)
test_QP1D5_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D5 = pd.concat([test_QP1D5_df1_1, test_QP1D5_df1_2, test_QP1D5_df1_3, test_QP1D5_df3, test_QP1D5_df4], axis=1)
test_QP1D5_onlyGhost = pd.concat([test_QP1D5_df3, test_QP1D5_df4], axis=1)


In [19]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D6_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D6_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D6_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D6_df2 = pd.DataFrame(columns=label_columns)
test_QP1D6_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D6_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D6_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D6_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D6_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D6_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D6_df1_1 = pd.concat([test_QP1D6_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D6_df1_2 = pd.concat([test_QP1D6_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D6_df1_3 = pd.concat([test_QP1D6_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D6_df2 = pd.concat([test_QP1D6_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D6_final_QP = extract_finalQP(test_QP1D6_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D6_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D6_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D6_df3 = pd.concat([test_QP1D6_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D6_df4 = pd.concat([test_QP1D6_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D6_df5 = pd.concat([test_QP1D6_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D6_df6 = pd.concat([test_QP1D6_df6, pd.DataFrame({"FINAL_QP": [test_QP1D6_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D6_df1_1.reset_index(drop=True, inplace=True)
test_QP1D6_df1_2.reset_index(drop=True, inplace=True)
test_QP1D6_df1_3.reset_index(drop=True, inplace=True)
test_QP1D6_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D6 = pd.concat([test_QP1D6_df1_1, test_QP1D6_df1_2, test_QP1D6_df1_3, test_QP1D6_df3, test_QP1D6_df4], axis=1)
test_QP1D6_onlyGhost = pd.concat([test_QP1D6_df3, test_QP1D6_df4], axis=1)


In [20]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D8_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D8_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D8_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D8_df2 = pd.DataFrame(columns=label_columns)
test_QP1D8_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D8_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D8_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D8_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D8_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D8_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D8_df1_1 = pd.concat([test_QP1D8_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D8_df1_2 = pd.concat([test_QP1D8_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D8_df1_3 = pd.concat([test_QP1D8_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D8_df2 = pd.concat([test_QP1D8_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D8_final_QP = extract_finalQP(test_QP1D8_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D8_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D8_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D8_df3 = pd.concat([test_QP1D8_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D8_df4 = pd.concat([test_QP1D8_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D8_df5 = pd.concat([test_QP1D8_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D8_df6 = pd.concat([test_QP1D8_df6, pd.DataFrame({"FINAL_QP": [test_QP1D8_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D8_df1_1.reset_index(drop=True, inplace=True)
test_QP1D8_df1_2.reset_index(drop=True, inplace=True)
test_QP1D8_df1_3.reset_index(drop=True, inplace=True)
test_QP1D8_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D8 = pd.concat([test_QP1D8_df1_1, test_QP1D8_df1_2, test_QP1D8_df1_3, test_QP1D8_df3, test_QP1D8_df4], axis=1)
test_QP1D8_onlyGhost = pd.concat([test_QP1D8_df3, test_QP1D8_df4], axis=1)


In [21]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D9_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D9_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D9_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D9_df2 = pd.DataFrame(columns=label_columns)
test_QP1D9_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D9_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D9_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D9_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D9_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D9_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D9_df1_1 = pd.concat([test_QP1D9_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D9_df1_2 = pd.concat([test_QP1D9_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D9_df1_3 = pd.concat([test_QP1D9_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D9_df2 = pd.concat([test_QP1D9_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D9_final_QP = extract_finalQP(test_QP1D9_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D9_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D9_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D9_df3 = pd.concat([test_QP1D9_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D9_df4 = pd.concat([test_QP1D9_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D9_df5 = pd.concat([test_QP1D9_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D9_df6 = pd.concat([test_QP1D9_df6, pd.DataFrame({"FINAL_QP": [test_QP1D9_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D9_df1_1.reset_index(drop=True, inplace=True)
test_QP1D9_df1_2.reset_index(drop=True, inplace=True)
test_QP1D9_df1_3.reset_index(drop=True, inplace=True)
test_QP1D9_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D9 = pd.concat([test_QP1D9_df1_1, test_QP1D9_df1_2, test_QP1D9_df1_3, test_QP1D9_df3, test_QP1D9_df4], axis=1)
test_QP1D9_onlyGhost = pd.concat([test_QP1D9_df3, test_QP1D9_df4], axis=1)


In [22]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D10_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D10_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D10_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D10_df2 = pd.DataFrame(columns=label_columns)
test_QP1D10_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D10_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D10_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D10_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D10_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D10_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D10_df1_1 = pd.concat([test_QP1D10_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D10_df1_2 = pd.concat([test_QP1D10_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D10_df1_3 = pd.concat([test_QP1D10_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D10_df2 = pd.concat([test_QP1D10_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D10_final_QP = extract_finalQP(test_QP1D10_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D10_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D10_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D10_df3 = pd.concat([test_QP1D10_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D10_df4 = pd.concat([test_QP1D10_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D10_df5 = pd.concat([test_QP1D10_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D10_df6 = pd.concat([test_QP1D10_df6, pd.DataFrame({"FINAL_QP": [test_QP1D10_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D10_df1_1.reset_index(drop=True, inplace=True)
test_QP1D10_df1_2.reset_index(drop=True, inplace=True)
test_QP1D10_df1_3.reset_index(drop=True, inplace=True)
test_QP1D10_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D10 = pd.concat([test_QP1D10_df1_1, test_QP1D10_df1_2, test_QP1D10_df1_3, test_QP1D10_df3, test_QP1D10_df4], axis=1)
test_QP1D10_onlyGhost = pd.concat([test_QP1D10_df3, test_QP1D10_df4], axis=1)


In [23]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D11_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D11_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D11_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D11_df2 = pd.DataFrame(columns=label_columns)
test_QP1D11_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D11_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D11_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D11_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D11_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D11_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D11_df1_1 = pd.concat([test_QP1D11_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D11_df1_2 = pd.concat([test_QP1D11_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D11_df1_3 = pd.concat([test_QP1D11_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D11_df2 = pd.concat([test_QP1D11_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D11_final_QP = extract_finalQP(test_QP1D11_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D11_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D11_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D11_df3 = pd.concat([test_QP1D11_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D11_df4 = pd.concat([test_QP1D11_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D11_df5 = pd.concat([test_QP1D11_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D11_df6 = pd.concat([test_QP1D11_df6, pd.DataFrame({"FINAL_QP": [test_QP1D11_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D11_df1_1.reset_index(drop=True, inplace=True)
test_QP1D11_df1_2.reset_index(drop=True, inplace=True)
test_QP1D11_df1_3.reset_index(drop=True, inplace=True)
test_QP1D11_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D11 = pd.concat([test_QP1D11_df1_1, test_QP1D11_df1_2, test_QP1D11_df1_3, test_QP1D11_df3, test_QP1D11_df4], axis=1)
test_QP1D11_onlyGhost = pd.concat([test_QP1D11_df3, test_QP1D11_df4], axis=1)


In [24]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D12_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D12_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D12_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D12_df2 = pd.DataFrame(columns=label_columns)
test_QP1D12_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D12_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D12_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D12_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D12_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D12_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D12_df1_1 = pd.concat([test_QP1D12_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D12_df1_2 = pd.concat([test_QP1D12_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D12_df1_3 = pd.concat([test_QP1D12_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D12_df2 = pd.concat([test_QP1D12_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D12_final_QP = extract_finalQP(test_QP1D12_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D12_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D12_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D12_df3 = pd.concat([test_QP1D12_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D12_df4 = pd.concat([test_QP1D12_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D12_df5 = pd.concat([test_QP1D12_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D12_df6 = pd.concat([test_QP1D12_df6, pd.DataFrame({"FINAL_QP": [test_QP1D12_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D12_df1_1.reset_index(drop=True, inplace=True)
test_QP1D12_df1_2.reset_index(drop=True, inplace=True)
test_QP1D12_df1_3.reset_index(drop=True, inplace=True)
test_QP1D12_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D12 = pd.concat([test_QP1D12_df1_1, test_QP1D12_df1_2, test_QP1D12_df1_3, test_QP1D12_df3, test_QP1D12_df4], axis=1)
test_QP1D12_onlyGhost = pd.concat([test_QP1D12_df3, test_QP1D12_df4], axis=1)


In [25]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D13_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D13_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D13_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D13_df2 = pd.DataFrame(columns=label_columns)
test_QP1D13_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D13_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D13_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D13_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D13_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D13_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D13_df1_1 = pd.concat([test_QP1D13_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D13_df1_2 = pd.concat([test_QP1D13_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D13_df1_3 = pd.concat([test_QP1D13_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D13_df2 = pd.concat([test_QP1D13_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D13_final_QP = extract_finalQP(test_QP1D13_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D13_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D13_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D13_df3 = pd.concat([test_QP1D13_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D13_df4 = pd.concat([test_QP1D13_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D13_df5 = pd.concat([test_QP1D13_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D13_df6 = pd.concat([test_QP1D13_df6, pd.DataFrame({"FINAL_QP": [test_QP1D13_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D13_df1_1.reset_index(drop=True, inplace=True)
test_QP1D13_df1_2.reset_index(drop=True, inplace=True)
test_QP1D13_df1_3.reset_index(drop=True, inplace=True)
test_QP1D13_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D13 = pd.concat([test_QP1D13_df1_1, test_QP1D13_df1_2, test_QP1D13_df1_3, test_QP1D13_df3, test_QP1D13_df4], axis=1)
test_QP1D13_onlyGhost = pd.concat([test_QP1D13_df3, test_QP1D13_df4], axis=1)


In [26]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D14_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D14_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D14_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D14_df2 = pd.DataFrame(columns=label_columns)
test_QP1D14_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D14_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D14_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D14_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D14_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D14_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D14_df1_1 = pd.concat([test_QP1D14_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D14_df1_2 = pd.concat([test_QP1D14_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D14_df1_3 = pd.concat([test_QP1D14_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D14_df2 = pd.concat([test_QP1D14_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D14_final_QP = extract_finalQP(test_QP1D14_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D14_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D14_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D14_df3 = pd.concat([test_QP1D14_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D14_df4 = pd.concat([test_QP1D14_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D14_df5 = pd.concat([test_QP1D14_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D14_df6 = pd.concat([test_QP1D14_df6, pd.DataFrame({"FINAL_QP": [test_QP1D14_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D14_df1_1.reset_index(drop=True, inplace=True)
test_QP1D14_df1_2.reset_index(drop=True, inplace=True)
test_QP1D14_df1_3.reset_index(drop=True, inplace=True)
test_QP1D14_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D14 = pd.concat([test_QP1D14_df1_1, test_QP1D14_df1_2, test_QP1D14_df1_3, test_QP1D14_df3, test_QP1D14_df4], axis=1)
test_QP1D14_onlyGhost = pd.concat([test_QP1D14_df3, test_QP1D14_df4], axis=1)


In [27]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D15_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D15_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D15_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D15_df2 = pd.DataFrame(columns=label_columns)
test_QP1D15_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D15_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D15_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D15_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D15_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D15_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D15_df1_1 = pd.concat([test_QP1D15_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D15_df1_2 = pd.concat([test_QP1D15_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D15_df1_3 = pd.concat([test_QP1D15_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D15_df2 = pd.concat([test_QP1D15_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D15_final_QP = extract_finalQP(test_QP1D15_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D15_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D15_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D15_df3 = pd.concat([test_QP1D15_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D15_df4 = pd.concat([test_QP1D15_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D15_df5 = pd.concat([test_QP1D15_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D15_df6 = pd.concat([test_QP1D15_df6, pd.DataFrame({"FINAL_QP": [test_QP1D15_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D15_df1_1.reset_index(drop=True, inplace=True)
test_QP1D15_df1_2.reset_index(drop=True, inplace=True)
test_QP1D15_df1_3.reset_index(drop=True, inplace=True)
test_QP1D15_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D15 = pd.concat([test_QP1D15_df1_1, test_QP1D15_df1_2, test_QP1D15_df1_3, test_QP1D15_df3, test_QP1D15_df4], axis=1)
test_QP1D15_onlyGhost = pd.concat([test_QP1D15_df3, test_QP1D15_df4], axis=1)


In [28]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D16_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D16_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D16_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D16_df2 = pd.DataFrame(columns=label_columns)
test_QP1D16_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D16_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D16_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D16_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D16_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D16_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D16_df1_1 = pd.concat([test_QP1D16_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D16_df1_2 = pd.concat([test_QP1D16_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D16_df1_3 = pd.concat([test_QP1D16_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D16_df2 = pd.concat([test_QP1D16_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D16_final_QP = extract_finalQP(test_QP1D16_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D16_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D16_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D16_df3 = pd.concat([test_QP1D16_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D16_df4 = pd.concat([test_QP1D16_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D16_df5 = pd.concat([test_QP1D16_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D16_df6 = pd.concat([test_QP1D16_df6, pd.DataFrame({"FINAL_QP": [test_QP1D16_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D16_df1_1.reset_index(drop=True, inplace=True)
test_QP1D16_df1_2.reset_index(drop=True, inplace=True)
test_QP1D16_df1_3.reset_index(drop=True, inplace=True)
test_QP1D16_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D16 = pd.concat([test_QP1D16_df1_1, test_QP1D16_df1_2, test_QP1D16_df1_3, test_QP1D16_df3, test_QP1D16_df4], axis=1)
test_QP1D16_onlyGhost = pd.concat([test_QP1D16_df3, test_QP1D16_df4], axis=1)


In [29]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D18_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D18_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D18_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D18_df2 = pd.DataFrame(columns=label_columns)
test_QP1D18_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D18_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D18_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D18_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D18_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D18_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D18_df1_1 = pd.concat([test_QP1D18_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D18_df1_2 = pd.concat([test_QP1D18_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D18_df1_3 = pd.concat([test_QP1D18_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D18_df2 = pd.concat([test_QP1D18_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D18_final_QP = extract_finalQP(test_QP1D18_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D18_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D18_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D18_df3 = pd.concat([test_QP1D18_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D18_df4 = pd.concat([test_QP1D18_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D18_df5 = pd.concat([test_QP1D18_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D18_df6 = pd.concat([test_QP1D18_df6, pd.DataFrame({"FINAL_QP": [test_QP1D18_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D18_df1_1.reset_index(drop=True, inplace=True)
test_QP1D18_df1_2.reset_index(drop=True, inplace=True)
test_QP1D18_df1_3.reset_index(drop=True, inplace=True)
test_QP1D18_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D18 = pd.concat([test_QP1D18_df1_1, test_QP1D18_df1_2, test_QP1D18_df1_3, test_QP1D18_df3, test_QP1D18_df4], axis=1)
test_QP1D18_onlyGhost = pd.concat([test_QP1D18_df3, test_QP1D18_df4], axis=1)


In [30]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D19_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D19_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D19_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D19_df2 = pd.DataFrame(columns=label_columns)
test_QP1D19_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D19_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D19_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D19_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D19_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D19_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D19_df1_1 = pd.concat([test_QP1D19_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D19_df1_2 = pd.concat([test_QP1D19_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D19_df1_3 = pd.concat([test_QP1D19_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D19_df2 = pd.concat([test_QP1D19_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D19_final_QP = extract_finalQP(test_QP1D19_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D19_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D19_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D19_df3 = pd.concat([test_QP1D19_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D19_df4 = pd.concat([test_QP1D19_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D19_df5 = pd.concat([test_QP1D19_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D19_df6 = pd.concat([test_QP1D19_df6, pd.DataFrame({"FINAL_QP": [test_QP1D19_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D19_df1_1.reset_index(drop=True, inplace=True)
test_QP1D19_df1_2.reset_index(drop=True, inplace=True)
test_QP1D19_df1_3.reset_index(drop=True, inplace=True)
test_QP1D19_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D19 = pd.concat([test_QP1D19_df1_1, test_QP1D19_df1_2, test_QP1D19_df1_3, test_QP1D19_df3, test_QP1D19_df4], axis=1)
test_QP1D19_onlyGhost = pd.concat([test_QP1D19_df3, test_QP1D19_df4], axis=1)


In [31]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D20_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D20_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D20_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D20_df2 = pd.DataFrame(columns=label_columns)
test_QP1D20_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D20_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D20_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D20_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D20_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D20_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D20_df1_1 = pd.concat([test_QP1D20_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D20_df1_2 = pd.concat([test_QP1D20_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D20_df1_3 = pd.concat([test_QP1D20_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D20_df2 = pd.concat([test_QP1D20_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D20_final_QP = extract_finalQP(test_QP1D20_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D20_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D20_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D20_df3 = pd.concat([test_QP1D20_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D20_df4 = pd.concat([test_QP1D20_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D20_df5 = pd.concat([test_QP1D20_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D20_df6 = pd.concat([test_QP1D20_df6, pd.DataFrame({"FINAL_QP": [test_QP1D20_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D20_df1_1.reset_index(drop=True, inplace=True)
test_QP1D20_df1_2.reset_index(drop=True, inplace=True)
test_QP1D20_df1_3.reset_index(drop=True, inplace=True)
test_QP1D20_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D20 = pd.concat([test_QP1D20_df1_1, test_QP1D20_df1_2, test_QP1D20_df1_3, test_QP1D20_df3, test_QP1D20_df4], axis=1)
test_QP1D20_onlyGhost = pd.concat([test_QP1D20_df3, test_QP1D20_df4], axis=1)


In [32]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D21_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D21_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D21_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D21_df2 = pd.DataFrame(columns=label_columns)
test_QP1D21_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D21_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D21_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D21_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D21_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D21_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D21_df1_1 = pd.concat([test_QP1D21_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D21_df1_2 = pd.concat([test_QP1D21_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D21_df1_3 = pd.concat([test_QP1D21_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D21_df2 = pd.concat([test_QP1D21_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D21_final_QP = extract_finalQP(test_QP1D21_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D21_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D21_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D21_df3 = pd.concat([test_QP1D21_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D21_df4 = pd.concat([test_QP1D21_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D21_df5 = pd.concat([test_QP1D21_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D21_df6 = pd.concat([test_QP1D21_df6, pd.DataFrame({"FINAL_QP": [test_QP1D21_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D21_df1_1.reset_index(drop=True, inplace=True)
test_QP1D21_df1_2.reset_index(drop=True, inplace=True)
test_QP1D21_df1_3.reset_index(drop=True, inplace=True)
test_QP1D21_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D21 = pd.concat([test_QP1D21_df1_1, test_QP1D21_df1_2, test_QP1D21_df1_3, test_QP1D21_df3, test_QP1D21_df4], axis=1)
test_QP1D21_onlyGhost = pd.concat([test_QP1D21_df3, test_QP1D21_df4], axis=1)


In [33]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D22_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D22_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D22_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D22_df2 = pd.DataFrame(columns=label_columns)
test_QP1D22_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D22_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D22_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D22_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D22_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D22_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D22_df1_1 = pd.concat([test_QP1D22_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D22_df1_2 = pd.concat([test_QP1D22_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D22_df1_3 = pd.concat([test_QP1D22_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D22_df2 = pd.concat([test_QP1D22_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D22_final_QP = extract_finalQP(test_QP1D22_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D22_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D22_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D22_df3 = pd.concat([test_QP1D22_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D22_df4 = pd.concat([test_QP1D22_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D22_df5 = pd.concat([test_QP1D22_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D22_df6 = pd.concat([test_QP1D22_df6, pd.DataFrame({"FINAL_QP": [test_QP1D22_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D22_df1_1.reset_index(drop=True, inplace=True)
test_QP1D22_df1_2.reset_index(drop=True, inplace=True)
test_QP1D22_df1_3.reset_index(drop=True, inplace=True)
test_QP1D22_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D22 = pd.concat([test_QP1D22_df1_1, test_QP1D22_df1_2, test_QP1D22_df1_3, test_QP1D22_df3, test_QP1D22_df4], axis=1)
test_QP1D22_onlyGhost = pd.concat([test_QP1D22_df3, test_QP1D22_df4], axis=1)


In [34]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D23_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D23_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D23_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D23_df2 = pd.DataFrame(columns=label_columns)
test_QP1D23_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D23_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D23_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D23_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D23_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D23_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D23_df1_1 = pd.concat([test_QP1D23_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D23_df1_2 = pd.concat([test_QP1D23_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D23_df1_3 = pd.concat([test_QP1D23_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D23_df2 = pd.concat([test_QP1D23_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D23_final_QP = extract_finalQP(test_QP1D23_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D23_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D23_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D23_df3 = pd.concat([test_QP1D23_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D23_df4 = pd.concat([test_QP1D23_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D23_df5 = pd.concat([test_QP1D23_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D23_df6 = pd.concat([test_QP1D23_df6, pd.DataFrame({"FINAL_QP": [test_QP1D23_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D23_df1_1.reset_index(drop=True, inplace=True)
test_QP1D23_df1_2.reset_index(drop=True, inplace=True)
test_QP1D23_df1_3.reset_index(drop=True, inplace=True)
test_QP1D23_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D23 = pd.concat([test_QP1D23_df1_1, test_QP1D23_df1_2, test_QP1D23_df1_3, test_QP1D23_df3, test_QP1D23_df4], axis=1)
test_QP1D23_onlyGhost = pd.concat([test_QP1D23_df3, test_QP1D23_df4], axis=1)


In [35]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D24_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D24_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D24_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D24_df2 = pd.DataFrame(columns=label_columns)
test_QP1D24_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D24_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D24_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D24_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D24_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D24_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D24_df1_1 = pd.concat([test_QP1D24_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D24_df1_2 = pd.concat([test_QP1D24_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D24_df1_3 = pd.concat([test_QP1D24_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D24_df2 = pd.concat([test_QP1D24_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D24_final_QP = extract_finalQP(test_QP1D24_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D24_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D24_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D24_df3 = pd.concat([test_QP1D24_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D24_df4 = pd.concat([test_QP1D24_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D24_df5 = pd.concat([test_QP1D24_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D24_df6 = pd.concat([test_QP1D24_df6, pd.DataFrame({"FINAL_QP": [test_QP1D24_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D24_df1_1.reset_index(drop=True, inplace=True)
test_QP1D24_df1_2.reset_index(drop=True, inplace=True)
test_QP1D24_df1_3.reset_index(drop=True, inplace=True)
test_QP1D24_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D24 = pd.concat([test_QP1D24_df1_1, test_QP1D24_df1_2, test_QP1D24_df1_3, test_QP1D24_df3, test_QP1D24_df4], axis=1)
test_QP1D24_onlyGhost = pd.concat([test_QP1D24_df3, test_QP1D24_df4], axis=1)


In [36]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D25_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D25_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D25_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D25_df2 = pd.DataFrame(columns=label_columns)
test_QP1D25_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D25_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D25_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D25_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D25_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D25_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D25_df1_1 = pd.concat([test_QP1D25_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D25_df1_2 = pd.concat([test_QP1D25_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D25_df1_3 = pd.concat([test_QP1D25_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D25_df2 = pd.concat([test_QP1D25_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D25_final_QP = extract_finalQP(test_QP1D25_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D25_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D25_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D25_df3 = pd.concat([test_QP1D25_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D25_df4 = pd.concat([test_QP1D25_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D25_df5 = pd.concat([test_QP1D25_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D25_df6 = pd.concat([test_QP1D25_df6, pd.DataFrame({"FINAL_QP": [test_QP1D25_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D25_df1_1.reset_index(drop=True, inplace=True)
test_QP1D25_df1_2.reset_index(drop=True, inplace=True)
test_QP1D25_df1_3.reset_index(drop=True, inplace=True)
test_QP1D25_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D25 = pd.concat([test_QP1D25_df1_1, test_QP1D25_df1_2, test_QP1D25_df1_3, test_QP1D25_df3, test_QP1D25_df4], axis=1)
test_QP1D25_onlyGhost = pd.concat([test_QP1D25_df3, test_QP1D25_df4], axis=1)


In [37]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D26_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D26_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D26_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D26_df2 = pd.DataFrame(columns=label_columns)
test_QP1D26_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D26_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D26_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D26_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D26_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D26_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D26_df1_1 = pd.concat([test_QP1D26_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D26_df1_2 = pd.concat([test_QP1D26_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D26_df1_3 = pd.concat([test_QP1D26_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D26_df2 = pd.concat([test_QP1D26_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D26_final_QP = extract_finalQP(test_QP1D26_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D26_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D26_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D26_df3 = pd.concat([test_QP1D26_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D26_df4 = pd.concat([test_QP1D26_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D26_df5 = pd.concat([test_QP1D26_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D26_df6 = pd.concat([test_QP1D26_df6, pd.DataFrame({"FINAL_QP": [test_QP1D26_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D26_df1_1.reset_index(drop=True, inplace=True)
test_QP1D26_df1_2.reset_index(drop=True, inplace=True)
test_QP1D26_df1_3.reset_index(drop=True, inplace=True)
test_QP1D26_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D26 = pd.concat([test_QP1D26_df1_1, test_QP1D26_df1_2, test_QP1D26_df1_3, test_QP1D26_df3, test_QP1D26_df4], axis=1)
test_QP1D26_onlyGhost = pd.concat([test_QP1D26_df3, test_QP1D26_df4], axis=1)


In [38]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D27_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D27_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D27_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D27_df2 = pd.DataFrame(columns=label_columns)
test_QP1D27_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D27_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D27_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D27_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D27_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D27_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D27_df1_1 = pd.concat([test_QP1D27_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D27_df1_2 = pd.concat([test_QP1D27_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D27_df1_3 = pd.concat([test_QP1D27_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D27_df2 = pd.concat([test_QP1D27_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D27_final_QP = extract_finalQP(test_QP1D27_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D27_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D27_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D27_df3 = pd.concat([test_QP1D27_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D27_df4 = pd.concat([test_QP1D27_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D27_df5 = pd.concat([test_QP1D27_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D27_df6 = pd.concat([test_QP1D27_df6, pd.DataFrame({"FINAL_QP": [test_QP1D27_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D27_df1_1.reset_index(drop=True, inplace=True)
test_QP1D27_df1_2.reset_index(drop=True, inplace=True)
test_QP1D27_df1_3.reset_index(drop=True, inplace=True)
test_QP1D27_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D27 = pd.concat([test_QP1D27_df1_1, test_QP1D27_df1_2, test_QP1D27_df1_3, test_QP1D27_df3, test_QP1D27_df4], axis=1)
test_QP1D27_onlyGhost = pd.concat([test_QP1D27_df3, test_QP1D27_df4], axis=1)


In [39]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D29_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D29_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D29_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D29_df2 = pd.DataFrame(columns=label_columns)
test_QP1D29_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D29_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D29_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D29_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D29_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D29_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D29_df1_1 = pd.concat([test_QP1D29_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D29_df1_2 = pd.concat([test_QP1D29_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D29_df1_3 = pd.concat([test_QP1D29_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D29_df2 = pd.concat([test_QP1D29_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D29_final_QP = extract_finalQP(test_QP1D29_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D29_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D29_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D29_df3 = pd.concat([test_QP1D29_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D29_df4 = pd.concat([test_QP1D29_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D29_df5 = pd.concat([test_QP1D29_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D29_df6 = pd.concat([test_QP1D29_df6, pd.DataFrame({"FINAL_QP": [test_QP1D29_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D29_df1_1.reset_index(drop=True, inplace=True)
test_QP1D29_df1_2.reset_index(drop=True, inplace=True)
test_QP1D29_df1_3.reset_index(drop=True, inplace=True)
test_QP1D29_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D29 = pd.concat([test_QP1D29_df1_1, test_QP1D29_df1_2, test_QP1D29_df1_3, test_QP1D29_df3, test_QP1D29_df4], axis=1)
test_QP1D29_onlyGhost = pd.concat([test_QP1D29_df3, test_QP1D29_df4], axis=1)


In [40]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D30_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D30_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D30_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D30_df2 = pd.DataFrame(columns=label_columns)
test_QP1D30_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D30_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D30_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D30_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D30_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D30_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D30_df1_1 = pd.concat([test_QP1D30_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D30_df1_2 = pd.concat([test_QP1D30_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D30_df1_3 = pd.concat([test_QP1D30_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D30_df2 = pd.concat([test_QP1D30_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D30_final_QP = extract_finalQP(test_QP1D30_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D30_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D30_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D30_df3 = pd.concat([test_QP1D30_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D30_df4 = pd.concat([test_QP1D30_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D30_df5 = pd.concat([test_QP1D30_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D30_df6 = pd.concat([test_QP1D30_df6, pd.DataFrame({"FINAL_QP": [test_QP1D30_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D30_df1_1.reset_index(drop=True, inplace=True)
test_QP1D30_df1_2.reset_index(drop=True, inplace=True)
test_QP1D30_df1_3.reset_index(drop=True, inplace=True)
test_QP1D30_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D30 = pd.concat([test_QP1D30_df1_1, test_QP1D30_df1_2, test_QP1D30_df1_3, test_QP1D30_df3, test_QP1D30_df4], axis=1)
test_QP1D30_onlyGhost = pd.concat([test_QP1D30_df3, test_QP1D30_df4], axis=1)


In [41]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D34_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D34_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D34_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D34_df2 = pd.DataFrame(columns=label_columns)
test_QP1D34_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D34_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D34_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D34_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D34_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D34_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D34_df1_1 = pd.concat([test_QP1D34_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D34_df1_2 = pd.concat([test_QP1D34_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D34_df1_3 = pd.concat([test_QP1D34_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D34_df2 = pd.concat([test_QP1D34_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D34_final_QP = extract_finalQP(test_QP1D34_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D34_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D34_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D34_df3 = pd.concat([test_QP1D34_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D34_df4 = pd.concat([test_QP1D34_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D34_df5 = pd.concat([test_QP1D34_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D34_df6 = pd.concat([test_QP1D34_df6, pd.DataFrame({"FINAL_QP": [test_QP1D34_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D34_df1_1.reset_index(drop=True, inplace=True)
test_QP1D34_df1_2.reset_index(drop=True, inplace=True)
test_QP1D34_df1_3.reset_index(drop=True, inplace=True)
test_QP1D34_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D34 = pd.concat([test_QP1D34_df1_1, test_QP1D34_df1_2, test_QP1D34_df1_3, test_QP1D34_df3, test_QP1D34_df4], axis=1)
test_QP1D34_onlyGhost = pd.concat([test_QP1D34_df3, test_QP1D34_df4], axis=1)


In [42]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D35_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D35_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D35_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D35_df2 = pd.DataFrame(columns=label_columns)
test_QP1D35_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D35_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D35_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D35_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D35_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D35_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D35_df1_1 = pd.concat([test_QP1D35_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D35_df1_2 = pd.concat([test_QP1D35_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D35_df1_3 = pd.concat([test_QP1D35_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D35_df2 = pd.concat([test_QP1D35_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D35_final_QP = extract_finalQP(test_QP1D35_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D35_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D35_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D35_df3 = pd.concat([test_QP1D35_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D35_df4 = pd.concat([test_QP1D35_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D35_df5 = pd.concat([test_QP1D35_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D35_df6 = pd.concat([test_QP1D35_df6, pd.DataFrame({"FINAL_QP": [test_QP1D35_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D35_df1_1.reset_index(drop=True, inplace=True)
test_QP1D35_df1_2.reset_index(drop=True, inplace=True)
test_QP1D35_df1_3.reset_index(drop=True, inplace=True)
test_QP1D35_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D35 = pd.concat([test_QP1D35_df1_1, test_QP1D35_df1_2, test_QP1D35_df1_3, test_QP1D35_df3, test_QP1D35_df4], axis=1)
test_QP1D35_onlyGhost = pd.concat([test_QP1D35_df3, test_QP1D35_df4], axis=1)


In [43]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D40_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D40_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D40_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D40_df2 = pd.DataFrame(columns=label_columns)
test_QP1D40_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D40_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D40_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D40_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D40_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D40_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D40_df1_1 = pd.concat([test_QP1D40_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D40_df1_2 = pd.concat([test_QP1D40_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D40_df1_3 = pd.concat([test_QP1D40_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D40_df2 = pd.concat([test_QP1D40_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D40_final_QP = extract_finalQP(test_QP1D40_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D40_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D40_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D40_df3 = pd.concat([test_QP1D40_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D40_df4 = pd.concat([test_QP1D40_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D40_df5 = pd.concat([test_QP1D40_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D40_df6 = pd.concat([test_QP1D40_df6, pd.DataFrame({"FINAL_QP": [test_QP1D40_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D40_df1_1.reset_index(drop=True, inplace=True)
test_QP1D40_df1_2.reset_index(drop=True, inplace=True)
test_QP1D40_df1_3.reset_index(drop=True, inplace=True)
test_QP1D40_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D40 = pd.concat([test_QP1D40_df1_1, test_QP1D40_df1_2, test_QP1D40_df1_3, test_QP1D40_df3, test_QP1D40_df4], axis=1)
test_QP1D40_onlyGhost = pd.concat([test_QP1D40_df3, test_QP1D40_df4], axis=1)


In [44]:
# 列名をリストにまとめる
pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
              "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

# luminance_columns = ["LU1_0", "LU1_1", "LU1_9", "LU1_10", "LU1_11", "LU1_25", "LU1_26", "LU1_27", 
#                      "LU2_0", "LU2_1", "LU2_9", "LU2_10", "LU2_11", "LU2_25", "LU2_26", "LU2_27"]

luminance_columns = ["LU1_0", "LU1_1", "LU1_10", "LU1_26",
                     "LU2_0", "LU2_1", "LU2_10", "LU2_26"]

chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                       "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]

label_columns = ["LABEL"]
mae1_columns = [f"MAE1_{i}" for i in range(52)]
mae2_columns = [f"MAE2_{i}" for i in range(52)]
mae_columns = ["MAE"]
final_qp_columns = ["FINAL_QP"]

# データフレームを初期化
test_QP1D45_df1_1 = pd.DataFrame(columns=pu_columns)
test_QP1D45_df1_2 = pd.DataFrame(columns=luminance_columns)
test_QP1D45_df1_3 = pd.DataFrame(columns=chrominance_columns)
test_QP1D45_df2 = pd.DataFrame(columns=label_columns)
test_QP1D45_df3 = pd.DataFrame(columns=mae1_columns)
test_QP1D45_df4 = pd.DataFrame(columns=mae2_columns)
test_QP1D45_df5 = pd.DataFrame(columns=mae_columns)
test_QP1D45_df6 = pd.DataFrame(columns=final_qp_columns)

scaler = MinMaxScaler()

for path1, path2, path3, path4 in test_csv_QP1D45_list:
    label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
    test_QP1D45_pkl_list = [path2, path4]
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path3)
    
    pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
    # lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
    # lu_values = lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
    
    lu_values_10_1 = [df1.loc[i, "luminance_counts"] for i in [9,10,11]]
    lu_values_10_2 = [df2.loc[i, "luminance_counts"] for i in [9,10,11]]
    
    lu_values_26_1 = [df1.loc[i, "luminance_counts"] for i in [25,26,27]] 
    lu_values_26_2 = [df2.loc[i, "luminance_counts"] for i in [25,26,27]]

    average_10_1 = np.mean(lu_values_10_1)
    average_10_2 = np.mean(lu_values_10_2)
    average_26_1 = np.mean(lu_values_26_1)
    average_26_2 = np.mean(lu_values_26_2)
    
    
    lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_1)] + [int(average_26_1)] + [df2.loc[i, "luminance_counts"] for i in [0,1]] + [int(average_10_2)] + [int(average_26_2)]
    ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]

    test_QP1D45_df1_1 = pd.concat([test_QP1D45_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
    test_QP1D45_df1_2 = pd.concat([test_QP1D45_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
    test_QP1D45_df1_3 = pd.concat([test_QP1D45_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
    # label_columnsの値を取得
    test_QP1D45_df2 = pd.concat([test_QP1D45_df2, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

    test_QP1D45_final_QP = extract_finalQP(test_QP1D45_pkl_list[0])

    # MAEの値を取得
    mae_d1, mae_d1_tensor = calculate_mae(test_QP1D45_pkl_list[0])
    mae_d2, _ = calculate_mae(test_QP1D45_pkl_list[1])


    # mae1_columnsの値を取得
    test_QP1D45_df3 = pd.concat([test_QP1D45_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)

    # mae2_columnsの値を取得
    test_QP1D45_df4 = pd.concat([test_QP1D45_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
    
    # mae_columnsの値を取得
    test_QP1D45_df5 = pd.concat([test_QP1D45_df5, pd.DataFrame({"MAE": [mae_d1_tensor]})], ignore_index=True)

    # final_qp_columnsの値を取得
    test_QP1D45_df6 = pd.concat([test_QP1D45_df6, pd.DataFrame({"FINAL_QP": [test_QP1D45_final_QP]})], ignore_index=True)


# インデックスをリセット
test_QP1D45_df1_1.reset_index(drop=True, inplace=True)
test_QP1D45_df1_2.reset_index(drop=True, inplace=True)
test_QP1D45_df1_3.reset_index(drop=True, inplace=True)
test_QP1D45_df2.reset_index(drop=True, inplace=True)
    
# データフレームを結合
test_QP1D45 = pd.concat([test_QP1D45_df1_1, test_QP1D45_df1_2, test_QP1D45_df1_3, test_QP1D45_df3, test_QP1D45_df4], axis=1)
test_QP1D45_onlyGhost = pd.concat([test_QP1D45_df3, test_QP1D45_df4], axis=1)


In [45]:
# スケーラーを使って結合したデータをスケーリング
X_test_QP1D1 = scaler.fit_transform(test_QP1D1)
X_test_QP1D1_onlyGhost = scaler.fit_transform(test_QP1D1_onlyGhost)

X_test_QP1D3 = scaler.fit_transform(test_QP1D3)
X_test_QP1D3_onlyGhost = scaler.fit_transform(test_QP1D3_onlyGhost)

X_test_QP1D4 = scaler.fit_transform(test_QP1D4)
X_test_QP1D4_onlyGhost = scaler.fit_transform(test_QP1D4_onlyGhost)

X_test_QP1D5 = scaler.fit_transform(test_QP1D5)
X_test_QP1D5_onlyGhost = scaler.fit_transform(test_QP1D5_onlyGhost)

X_test_QP1D6 = scaler.fit_transform(test_QP1D6)
X_test_QP1D6_onlyGhost = scaler.fit_transform(test_QP1D6_onlyGhost)

X_test_QP1D8 = scaler.fit_transform(test_QP1D8)
X_test_QP1D8_onlyGhost = scaler.fit_transform(test_QP1D8_onlyGhost)

X_test_QP1D9 = scaler.fit_transform(test_QP1D9)
X_test_QP1D9_onlyGhost = scaler.fit_transform(test_QP1D9_onlyGhost)

X_test_QP1D10 = scaler.fit_transform(test_QP1D10)
X_test_QP1D10_onlyGhost = scaler.fit_transform(test_QP1D10_onlyGhost)

X_test_QP1D11 = scaler.fit_transform(test_QP1D11)
X_test_QP1D11_onlyGhost = scaler.fit_transform(test_QP1D11_onlyGhost)

X_test_QP1D12 = scaler.fit_transform(test_QP1D12)
X_test_QP1D12_onlyGhost = scaler.fit_transform(test_QP1D12_onlyGhost)

X_test_QP1D13 = scaler.fit_transform(test_QP1D13)
X_test_QP1D13_onlyGhost = scaler.fit_transform(test_QP1D13_onlyGhost)

X_test_QP1D14 = scaler.fit_transform(test_QP1D14)
X_test_QP1D14_onlyGhost = scaler.fit_transform(test_QP1D14_onlyGhost)

X_test_QP1D15 = scaler.fit_transform(test_QP1D15)
X_test_QP1D15_onlyGhost = scaler.fit_transform(test_QP1D15_onlyGhost)

X_test_QP1D16 = scaler.fit_transform(test_QP1D16)
X_test_QP1D16_onlyGhost = scaler.fit_transform(test_QP1D16_onlyGhost)

X_test_QP1D18 = scaler.fit_transform(test_QP1D18)
X_test_QP1D18_onlyGhost = scaler.fit_transform(test_QP1D18_onlyGhost)

X_test_QP1D19 = scaler.fit_transform(test_QP1D19)
X_test_QP1D19_onlyGhost = scaler.fit_transform(test_QP1D19_onlyGhost)

X_test_QP1D20 = scaler.fit_transform(test_QP1D20)
X_test_QP1D20_onlyGhost = scaler.fit_transform(test_QP1D20_onlyGhost)

X_test_QP1D21 = scaler.fit_transform(test_QP1D21)
X_test_QP1D21_onlyGhost = scaler.fit_transform(test_QP1D21_onlyGhost)

X_test_QP1D22 = scaler.fit_transform(test_QP1D22)
X_test_QP1D22_onlyGhost = scaler.fit_transform(test_QP1D22_onlyGhost)

X_test_QP1D23 = scaler.fit_transform(test_QP1D23)
X_test_QP1D23_onlyGhost = scaler.fit_transform(test_QP1D23_onlyGhost)

X_test_QP1D24 = scaler.fit_transform(test_QP1D24)
X_test_QP1D24_onlyGhost = scaler.fit_transform(test_QP1D24_onlyGhost)

X_test_QP1D25 = scaler.fit_transform(test_QP1D25)
X_test_QP1D25_onlyGhost = scaler.fit_transform(test_QP1D25_onlyGhost)

X_test_QP1D26 = scaler.fit_transform(test_QP1D26)
X_test_QP1D26_onlyGhost = scaler.fit_transform(test_QP1D26_onlyGhost)

X_test_QP1D27 = scaler.fit_transform(test_QP1D27)
X_test_QP1D27_onlyGhost = scaler.fit_transform(test_QP1D27_onlyGhost)

X_test_QP1D29 = scaler.fit_transform(test_QP1D29)
X_test_QP1D29_onlyGhost = scaler.fit_transform(test_QP1D29_onlyGhost)

X_test_QP1D30 = scaler.fit_transform(test_QP1D30)
X_test_QP1D30_onlyGhost = scaler.fit_transform(test_QP1D30_onlyGhost)

X_test_QP1D34 = scaler.fit_transform(test_QP1D34)
X_test_QP1D34_onlyGhost = scaler.fit_transform(test_QP1D34_onlyGhost)

X_test_QP1D35 = scaler.fit_transform(test_QP1D35)
X_test_QP1D35_onlyGhost = scaler.fit_transform(test_QP1D35_onlyGhost)

X_test_QP1D40 = scaler.fit_transform(test_QP1D40)
X_test_QP1D40_onlyGhost = scaler.fit_transform(test_QP1D40_onlyGhost)

X_test_QP1D45 = scaler.fit_transform(test_QP1D45)
X_test_QP1D45_onlyGhost = scaler.fit_transform(test_QP1D45_onlyGhost)


# pandasをndarrayに変換
MAE_QP1D1 = test_QP1D1_df5.values
FINAL_QP_QP1D1 = test_QP1D1_df6.values
Y_test_QP1D1 = test_QP1D1_df2['LABEL'].astype(int)

MAE_QP1D3 = test_QP1D3_df5.values
FINAL_QP_QP1D3 = test_QP1D3_df6.values
Y_test_QP1D3 = test_QP1D3_df2['LABEL'].astype(int)

MAE_QP1D4 = test_QP1D4_df5.values
FINAL_QP_QP1D4 = test_QP1D4_df6.values
Y_test_QP1D4 = test_QP1D4_df2['LABEL'].astype(int)

MAE_QP1D5 = test_QP1D5_df5.values
FINAL_QP_QP1D5 = test_QP1D5_df6.values
Y_test_QP1D5 = test_QP1D5_df2['LABEL'].astype(int)

MAE_QP1D6 = test_QP1D6_df5.values
FINAL_QP_QP1D6 = test_QP1D6_df6.values
Y_test_QP1D6 = test_QP1D6_df2['LABEL'].astype(int)

MAE_QP1D8 = test_QP1D8_df5.values
FINAL_QP_QP1D8 = test_QP1D8_df6.values
Y_test_QP1D8 = test_QP1D8_df2['LABEL'].astype(int)

MAE_QP1D9 = test_QP1D9_df5.values
FINAL_QP_QP1D9 = test_QP1D9_df6.values
Y_test_QP1D9 = test_QP1D9_df2['LABEL'].astype(int)

MAE_QP1D10 = test_QP1D10_df5.values
FINAL_QP_QP1D10 = test_QP1D10_df6.values
Y_test_QP1D10 = test_QP1D10_df2['LABEL'].astype(int)

MAE_QP1D11 = test_QP1D11_df5.values
FINAL_QP_QP1D11 = test_QP1D11_df6.values
Y_test_QP1D11 = test_QP1D11_df2['LABEL'].astype(int)

MAE_QP1D12 = test_QP1D12_df5.values
FINAL_QP_QP1D12 = test_QP1D12_df6.values
Y_test_QP1D12 = test_QP1D12_df2['LABEL'].astype(int)

MAE_QP1D13 = test_QP1D13_df5.values
FINAL_QP_QP1D13 = test_QP1D13_df6.values
Y_test_QP1D13 = test_QP1D13_df2['LABEL'].astype(int)

MAE_QP1D14 = test_QP1D14_df5.values
FINAL_QP_QP1D14 = test_QP1D14_df6.values
Y_test_QP1D14 = test_QP1D14_df2['LABEL'].astype(int)

MAE_QP1D15 = test_QP1D15_df5.values
FINAL_QP_QP1D15 = test_QP1D15_df6.values
Y_test_QP1D15 = test_QP1D15_df2['LABEL'].astype(int)

MAE_QP1D16 = test_QP1D16_df5.values
FINAL_QP_QP1D16 = test_QP1D16_df6.values
Y_test_QP1D16 = test_QP1D16_df2['LABEL'].astype(int)

MAE_QP1D18 = test_QP1D18_df5.values
FINAL_QP_QP1D18 = test_QP1D18_df6.values
Y_test_QP1D18 = test_QP1D18_df2['LABEL'].astype(int)

MAE_QP1D19 = test_QP1D19_df5.values
FINAL_QP_QP1D19 = test_QP1D19_df6.values
Y_test_QP1D19 = test_QP1D19_df2['LABEL'].astype(int)

MAE_QP1D20 = test_QP1D20_df5.values
FINAL_QP_QP1D20 = test_QP1D20_df6.values
Y_test_QP1D20 = test_QP1D20_df2['LABEL'].astype(int)

MAE_QP1D21 = test_QP1D21_df5.values
FINAL_QP_QP1D21 = test_QP1D21_df6.values
Y_test_QP1D21 = test_QP1D21_df2['LABEL'].astype(int)

MAE_QP1D22 = test_QP1D22_df5.values
FINAL_QP_QP1D22 = test_QP1D22_df6.values
Y_test_QP1D22 = test_QP1D22_df2['LABEL'].astype(int)

MAE_QP1D23 = test_QP1D23_df5.values
FINAL_QP_QP1D23 = test_QP1D23_df6.values
Y_test_QP1D23 = test_QP1D23_df2['LABEL'].astype(int)

MAE_QP1D24 = test_QP1D24_df5.values
FINAL_QP_QP1D24 = test_QP1D24_df6.values
Y_test_QP1D24 = test_QP1D24_df2['LABEL'].astype(int)

MAE_QP1D25 = test_QP1D25_df5.values
FINAL_QP_QP1D25 = test_QP1D25_df6.values
Y_test_QP1D25 = test_QP1D25_df2['LABEL'].astype(int)

MAE_QP1D26 = test_QP1D26_df5.values
FINAL_QP_QP1D26 = test_QP1D26_df6.values
Y_test_QP1D26 = test_QP1D26_df2['LABEL'].astype(int)

MAE_QP1D27 = test_QP1D27_df5.values
FINAL_QP_QP1D27 = test_QP1D27_df6.values
Y_test_QP1D27 = test_QP1D27_df2['LABEL'].astype(int)

MAE_QP1D29 = test_QP1D29_df5.values
FINAL_QP_QP1D29 = test_QP1D29_df6.values
Y_test_QP1D29 = test_QP1D29_df2['LABEL'].astype(int)

MAE_QP1D30 = test_QP1D30_df5.values
FINAL_QP_QP1D30 = test_QP1D30_df6.values
Y_test_QP1D30 = test_QP1D30_df2['LABEL'].astype(int)

MAE_QP1D34 = test_QP1D34_df5.values
FINAL_QP_QP1D34 = test_QP1D34_df6.values
Y_test_QP1D34 = test_QP1D34_df2['LABEL'].astype(int)

MAE_QP1D35 = test_QP1D35_df5.values
FINAL_QP_QP1D35 = test_QP1D35_df6.values
Y_test_QP1D35 = test_QP1D35_df2['LABEL'].astype(int)

MAE_QP1D40 = test_QP1D40_df5.values
FINAL_QP_QP1D40 = test_QP1D40_df6.values
Y_test_QP1D40 = test_QP1D40_df2['LABEL'].astype(int)

MAE_QP1D45 = test_QP1D45_df5.values
FINAL_QP_QP1D45 = test_QP1D45_df6.values
Y_test_QP1D45 = test_QP1D45_df2['LABEL'].astype(int)


datasets = [
    ('QP1D1', X_test_QP1D1, X_test_QP1D1_onlyGhost, MAE_QP1D1, FINAL_QP_QP1D1, Y_test_QP1D1),
    ('QP1D45', X_test_QP1D45, X_test_QP1D45_onlyGhost, MAE_QP1D45, FINAL_QP_QP1D45, Y_test_QP1D45),
]

for name, X, X_onlyGhost, MAE, FINAL_QP, Y in datasets:    
    # 出力
    print(f'Length of X_test_{name}: {len(X)}')
    print(f'Length of X_test_{name}_onlyGhost: {len(X_onlyGhost)}')
    print(f'Length of Y_test_{name}: {len(Y)}')
    print(f'Length of MAE_{name}: {len(MAE)}')
    print(f'Length of FINAL_QP_{name}: {len(FINAL_QP)}')
    print('-------------------------------------------')

Length of X_test_QP1D1: 20
Length of X_test_QP1D1_onlyGhost: 20
Length of Y_test_QP1D1: 20
Length of MAE_QP1D1: 20
Length of FINAL_QP_QP1D1: 20
-------------------------------------------
Length of X_test_QP1D45: 20
Length of X_test_QP1D45_onlyGhost: 20
Length of Y_test_QP1D45: 20
Length of MAE_QP1D45: 20
Length of FINAL_QP_QP1D45: 20
-------------------------------------------


In [46]:
# Cの範囲を指定
C_values = {'C': [0.01, 0.1, 1, 10, 100, 1000, 2000, 3000, 4000, 5000]}
# kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 結果のデータフレームを初期化
results = pd.DataFrame(columns=['RBF_QP1D1','LINEAR_QP1D1','RBF_ghost_QP1D1','LINEAR_ghost_QP1D1','QP1D1_old',
                                'RBF_QP1D3','LINEAR_QP1D3','RBF_ghost_QP1D3','LINEAR_ghost_QP1D3','QP1D3_old',
                                'RBF_QP1D4','LINEAR_QP1D4','RBF_ghost_QP1D4','LINEAR_ghost_QP1D4','QP1D4_old',
                                'RBF_QP1D5','LINEAR_QP1D5','RBF_ghost_QP1D5','LINEAR_ghost_QP1D5','QP1D5_old',
                                'RBF_QP1D6','LINEAR_QP1D6','RBF_ghost_QP1D6','LINEAR_ghost_QP1D6','QP1D6_old',
                                'RBF_QP1D8','LINEAR_QP1D8','RBF_ghost_QP1D8','LINEAR_ghost_QP1D8','QP1D8_old',
                                'RBF_QP1D9','LINEAR_QP1D9','RBF_ghost_QP1D9','LINEAR_ghost_QP1D9','QP1D9_old',
                                'RBF_QP1D10','LINEAR_QP1D10','RBF_ghost_QP1D10','LINEAR_ghost_QP1D10','QP1D10_old',
                                'RBF_QP1D11','LINEAR_QP1D11','RBF_ghost_QP1D11','LINEAR_ghost_QP1D11','QP1D11_old',
                                'RBF_QP1D12','LINEAR_QP1D12','RBF_ghost_QP1D12','LINEAR_ghost_QP1D12','QP1D12_old',
                                'RBF_QP1D13','LINEAR_QP1D13','RBF_ghost_QP1D13','LINEAR_ghost_QP1D13','QP1D13_old',
                                'RBF_QP1D14','LINEAR_QP1D14','RBF_ghost_QP1D14','LINEAR_ghost_QP1D14','QP1D14_old',
                                'RBF_QP1D15','LINEAR_QP1D15','RBF_ghost_QP1D15','LINEAR_ghost_QP1D15','QP1D15_old',
                                'RBF_QP1D16','LINEAR_QP1D16','RBF_ghost_QP1D16','LINEAR_ghost_QP1D16','QP1D16_old',
                                'RBF_QP1D18','LINEAR_QP1D18','RBF_ghost_QP1D18','LINEAR_ghost_QP1D18','QP1D18_old',
                                'RBF_QP1D19','LINEAR_QP1D19','RBF_ghost_QP1D19','LINEAR_ghost_QP1D19','QP1D19_old',
                                'RBF_QP1D20','LINEAR_QP1D20','RBF_ghost_QP1D20','LINEAR_ghost_QP1D20','QP1D20_old',
                                'RBF_QP1D21','LINEAR_QP1D21','RBF_ghost_QP1D21','LINEAR_ghost_QP1D21','QP1D21_old',
                                'RBF_QP1D22','LINEAR_QP1D22','RBF_ghost_QP1D22','LINEAR_ghost_QP1D22','QP1D22_old',
                                'RBF_QP1D23','LINEAR_QP1D23','RBF_ghost_QP1D23','LINEAR_ghost_QP1D23','QP1D23_old',
                                'RBF_QP1D24','LINEAR_QP1D24','RBF_ghost_QP1D24','LINEAR_ghost_QP1D24','QP1D24_old',
                                'RBF_QP1D25','LINEAR_QP1D25','RBF_ghost_QP1D25','LINEAR_ghost_QP1D25','QP1D25_old',
                                'RBF_QP1D26','LINEAR_QP1D26','RBF_ghost_QP1D26','LINEAR_ghost_QP1D26','QP1D26_old',
                                'RBF_QP1D27','LINEAR_QP1D27','RBF_ghost_QP1D27','LINEAR_ghost_QP1D27','QP1D27_old',
                                'RBF_QP1D29','LINEAR_QP1D29','RBF_ghost_QP1D29','LINEAR_ghost_QP1D29','QP1D29_old',
                                'RBF_QP1D30','LINEAR_QP1D30','RBF_ghost_QP1D30','LINEAR_ghost_QP1D30','QP1D30_old',
                                'RBF_QP1D34','LINEAR_QP1D34','RBF_ghost_QP1D34','LINEAR_ghost_QP1D34','QP1D34_old',
                                'RBF_QP1D35','LINEAR_QP1D35','RBF_ghost_QP1D35','LINEAR_ghost_QP1D35','QP1D35_old',
                                'RBF_QP1D40','LINEAR_QP1D40','RBF_ghost_QP1D40','LINEAR_ghost_QP1D40','QP1D40_old',
                                'RBF_QP1D45','LINEAR_QP1D45','RBF_ghost_QP1D45','LINEAR_ghost_QP1D45','QP1D45_old'
                               ])


# 訓練データ
O_X_train, O_X_train_onlyGhost = X_train, X_train_onlyGhost
O_Y_train = Y_train

# テストデータ - 30パターン
O_X_test_QP1D1, O_X_test_QP1D1_onlyGhost = X_test_QP1D1, X_test_QP1D1_onlyGhost
O_Y_test_QP1D1 = Y_test_QP1D1

O_X_test_QP1D3, O_X_test_QP1D3_onlyGhost = X_test_QP1D3, X_test_QP1D3_onlyGhost
O_Y_test_QP1D3 = Y_test_QP1D3

O_X_test_QP1D4, O_X_test_QP1D4_onlyGhost = X_test_QP1D4, X_test_QP1D4_onlyGhost
O_Y_test_QP1D4 = Y_test_QP1D4

O_X_test_QP1D5, O_X_test_QP1D5_onlyGhost = X_test_QP1D5, X_test_QP1D5_onlyGhost
O_Y_test_QP1D5 = Y_test_QP1D5

O_X_test_QP1D6, O_X_test_QP1D6_onlyGhost = X_test_QP1D6, X_test_QP1D6_onlyGhost
O_Y_test_QP1D6 = Y_test_QP1D6

O_X_test_QP1D8, O_X_test_QP1D8_onlyGhost = X_test_QP1D8, X_test_QP1D8_onlyGhost
O_Y_test_QP1D8 = Y_test_QP1D8

O_X_test_QP1D9, O_X_test_QP1D9_onlyGhost = X_test_QP1D9, X_test_QP1D9_onlyGhost
O_Y_test_QP1D9 = Y_test_QP1D9

O_X_test_QP1D10, O_X_test_QP1D10_onlyGhost = X_test_QP1D10, X_test_QP1D10_onlyGhost
O_Y_test_QP1D10 = Y_test_QP1D10

O_X_test_QP1D11, O_X_test_QP1D11_onlyGhost = X_test_QP1D11, X_test_QP1D11_onlyGhost
O_Y_test_QP1D11 = Y_test_QP1D11

O_X_test_QP1D12, O_X_test_QP1D12_onlyGhost = X_test_QP1D12, X_test_QP1D12_onlyGhost
O_Y_test_QP1D12 = Y_test_QP1D12

O_X_test_QP1D13, O_X_test_QP1D13_onlyGhost = X_test_QP1D13, X_test_QP1D13_onlyGhost
O_Y_test_QP1D13 = Y_test_QP1D13

O_X_test_QP1D14, O_X_test_QP1D14_onlyGhost = X_test_QP1D14, X_test_QP1D14_onlyGhost
O_Y_test_QP1D14 = Y_test_QP1D14

O_X_test_QP1D15, O_X_test_QP1D15_onlyGhost = X_test_QP1D15, X_test_QP1D15_onlyGhost
O_Y_test_QP1D15 = Y_test_QP1D15

O_X_test_QP1D16, O_X_test_QP1D16_onlyGhost = X_test_QP1D16, X_test_QP1D16_onlyGhost
O_Y_test_QP1D16 = Y_test_QP1D16

O_X_test_QP1D18, O_X_test_QP1D18_onlyGhost = X_test_QP1D18, X_test_QP1D18_onlyGhost
O_Y_test_QP1D18 = Y_test_QP1D18

O_X_test_QP1D19, O_X_test_QP1D19_onlyGhost = X_test_QP1D19, X_test_QP1D19_onlyGhost
O_Y_test_QP1D19 = Y_test_QP1D19

O_X_test_QP1D20, O_X_test_QP1D20_onlyGhost = X_test_QP1D20, X_test_QP1D20_onlyGhost
O_Y_test_QP1D20 = Y_test_QP1D20

O_X_test_QP1D21, O_X_test_QP1D21_onlyGhost = X_test_QP1D21, X_test_QP1D21_onlyGhost
O_Y_test_QP1D21 = Y_test_QP1D21

O_X_test_QP1D22, O_X_test_QP1D22_onlyGhost = X_test_QP1D22, X_test_QP1D22_onlyGhost
O_Y_test_QP1D22 = Y_test_QP1D22

O_X_test_QP1D23, O_X_test_QP1D23_onlyGhost = X_test_QP1D23, X_test_QP1D23_onlyGhost
O_Y_test_QP1D23 = Y_test_QP1D23

O_X_test_QP1D24, O_X_test_QP1D24_onlyGhost = X_test_QP1D24, X_test_QP1D24_onlyGhost
O_Y_test_QP1D24 = Y_test_QP1D24

O_X_test_QP1D25, O_X_test_QP1D25_onlyGhost = X_test_QP1D25, X_test_QP1D25_onlyGhost
O_Y_test_QP1D25 = Y_test_QP1D25

O_X_test_QP1D26, O_X_test_QP1D26_onlyGhost = X_test_QP1D26, X_test_QP1D26_onlyGhost
O_Y_test_QP1D26 = Y_test_QP1D26

O_X_test_QP1D27, O_X_test_QP1D27_onlyGhost = X_test_QP1D27, X_test_QP1D27_onlyGhost
O_Y_test_QP1D27 = Y_test_QP1D27

O_X_test_QP1D29, O_X_test_QP1D29_onlyGhost = X_test_QP1D29, X_test_QP1D29_onlyGhost
O_Y_test_QP1D29 = Y_test_QP1D29

O_X_test_QP1D30, O_X_test_QP1D30_onlyGhost = X_test_QP1D30, X_test_QP1D30_onlyGhost
O_Y_test_QP1D30 = Y_test_QP1D30

O_X_test_QP1D34, O_X_test_QP1D34_onlyGhost = X_test_QP1D34, X_test_QP1D34_onlyGhost
O_Y_test_QP1D34 = Y_test_QP1D34

O_X_test_QP1D35, O_X_test_QP1D35_onlyGhost = X_test_QP1D35, X_test_QP1D35_onlyGhost
O_Y_test_QP1D35 = Y_test_QP1D35

O_X_test_QP1D40, O_X_test_QP1D40_onlyGhost = X_test_QP1D40, X_test_QP1D40_onlyGhost
O_Y_test_QP1D40 = Y_test_QP1D40

O_X_test_QP1D45, O_X_test_QP1D45_onlyGhost = X_test_QP1D45, X_test_QP1D45_onlyGhost
O_Y_test_QP1D45 = Y_test_QP1D45

# 閾値用テストデータ - 30パターン
O_QP1D1_test_old, O_QP1D1_test_final_QP = MAE_QP1D1, FINAL_QP_QP1D1
O_QP1D3_test_old, O_QP1D3_test_final_QP = MAE_QP1D3, FINAL_QP_QP1D3
O_QP1D4_test_old, O_QP1D4_test_final_QP = MAE_QP1D4, FINAL_QP_QP1D4
O_QP1D5_test_old, O_QP1D5_test_final_QP = MAE_QP1D5, FINAL_QP_QP1D5
O_QP1D6_test_old, O_QP1D6_test_final_QP = MAE_QP1D6, FINAL_QP_QP1D6
O_QP1D8_test_old, O_QP1D8_test_final_QP = MAE_QP1D8, FINAL_QP_QP1D8
O_QP1D9_test_old, O_QP1D9_test_final_QP = MAE_QP1D9, FINAL_QP_QP1D9
O_QP1D10_test_old, O_QP1D10_test_final_QP = MAE_QP1D10, FINAL_QP_QP1D10
O_QP1D11_test_old, O_QP1D11_test_final_QP = MAE_QP1D11, FINAL_QP_QP1D11
O_QP1D12_test_old, O_QP1D12_test_final_QP = MAE_QP1D12, FINAL_QP_QP1D12
O_QP1D13_test_old, O_QP1D13_test_final_QP = MAE_QP1D13, FINAL_QP_QP1D13
O_QP1D14_test_old, O_QP1D14_test_final_QP = MAE_QP1D14, FINAL_QP_QP1D14
O_QP1D15_test_old, O_QP1D15_test_final_QP = MAE_QP1D15, FINAL_QP_QP1D15
O_QP1D16_test_old, O_QP1D16_test_final_QP = MAE_QP1D16, FINAL_QP_QP1D16
O_QP1D18_test_old, O_QP1D18_test_final_QP = MAE_QP1D18, FINAL_QP_QP1D18
O_QP1D19_test_old, O_QP1D19_test_final_QP = MAE_QP1D19, FINAL_QP_QP1D19
O_QP1D20_test_old, O_QP1D20_test_final_QP = MAE_QP1D20, FINAL_QP_QP1D20
O_QP1D21_test_old, O_QP1D21_test_final_QP = MAE_QP1D21, FINAL_QP_QP1D21
O_QP1D22_test_old, O_QP1D22_test_final_QP = MAE_QP1D22, FINAL_QP_QP1D22
O_QP1D23_test_old, O_QP1D23_test_final_QP = MAE_QP1D23, FINAL_QP_QP1D23
O_QP1D24_test_old, O_QP1D24_test_final_QP = MAE_QP1D24, FINAL_QP_QP1D24
O_QP1D25_test_old, O_QP1D25_test_final_QP = MAE_QP1D25, FINAL_QP_QP1D25
O_QP1D26_test_old, O_QP1D26_test_final_QP = MAE_QP1D26, FINAL_QP_QP1D26
O_QP1D27_test_old, O_QP1D27_test_final_QP = MAE_QP1D27, FINAL_QP_QP1D27
O_QP1D29_test_old, O_QP1D29_test_final_QP = MAE_QP1D29, FINAL_QP_QP1D29
O_QP1D30_test_old, O_QP1D30_test_final_QP = MAE_QP1D30, FINAL_QP_QP1D30
O_QP1D34_test_old, O_QP1D34_test_final_QP = MAE_QP1D34, FINAL_QP_QP1D34
O_QP1D35_test_old, O_QP1D35_test_final_QP = MAE_QP1D35, FINAL_QP_QP1D35
O_QP1D40_test_old, O_QP1D40_test_final_QP = MAE_QP1D40, FINAL_QP_QP1D40
O_QP1D45_test_old, O_QP1D45_test_final_QP = MAE_QP1D45, FINAL_QP_QP1D45


# k-fold cross-validation
for fold, (train_val_ids, test_ids) in enumerate(kfold.split(O_X_train, O_Y_train)):
    print(f"<Fold-{fold+1}>")
    print()
    
    print(len(train_val_ids), len(test_ids))
    
    results_old = []
    
    # 全体を訓練ラベルと検証ラベルに分割
    X_train_val, _ = O_X_train[train_val_ids], O_X_train[test_ids]
    X_train_val_onlyGhost, _ = O_X_train_onlyGhost[train_val_ids], O_X_train_onlyGhost[test_ids]
    Y_train_val, _ = O_Y_train[train_val_ids], O_Y_train[test_ids]
    
    # 訓練・検証データ（ラベル）を訓練データ（ラベル）と検証データ（ラベル）に分割
    X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size=552, random_state=42)
    X_train_onlyGhost, X_val_onlyGhost, Y_train, Y_val = train_test_split(X_train_val_onlyGhost, Y_train_val, test_size=552, random_state=42)
    
    
    # テストデータ・ラベルの処理（RBFとLINEAR）- 30パターン
    X_test_QP1D1 = O_X_test_QP1D1
    X_test_QP1D1_onlyGhost = O_X_test_QP1D1_onlyGhost
    Y_test_QP1D1 = O_Y_test_QP1D1
    
    X_test_QP1D3 = O_X_test_QP1D3
    X_test_QP1D3_onlyGhost = O_X_test_QP1D3_onlyGhost
    Y_test_QP1D3 = O_Y_test_QP1D3
    
    X_test_QP1D4 = O_X_test_QP1D4
    X_test_QP1D4_onlyGhost = O_X_test_QP1D4_onlyGhost
    Y_test_QP1D4 = O_Y_test_QP1D4
    
    X_test_QP1D5 = O_X_test_QP1D5
    X_test_QP1D5_onlyGhost = O_X_test_QP1D5_onlyGhost
    Y_test_QP1D5 = O_Y_test_QP1D5
    
    X_test_QP1D6 = O_X_test_QP1D6
    X_test_QP1D6_onlyGhost = O_X_test_QP1D6_onlyGhost
    Y_test_QP1D6 = O_Y_test_QP1D6
    
    X_test_QP1D8 = O_X_test_QP1D8
    X_test_QP1D8_onlyGhost = O_X_test_QP1D8_onlyGhost
    Y_test_QP1D8 = O_Y_test_QP1D8
    
    X_test_QP1D9 = O_X_test_QP1D9
    X_test_QP1D9_onlyGhost = O_X_test_QP1D9_onlyGhost
    Y_test_QP1D9 = O_Y_test_QP1D9
    
    X_test_QP1D10 = O_X_test_QP1D10
    X_test_QP1D10_onlyGhost = O_X_test_QP1D10_onlyGhost
    Y_test_QP1D10 = O_Y_test_QP1D10
    
    X_test_QP1D11 = O_X_test_QP1D11
    X_test_QP1D11_onlyGhost = O_X_test_QP1D11_onlyGhost
    Y_test_QP1D11 = O_Y_test_QP1D11
    
    X_test_QP1D12 = O_X_test_QP1D12
    X_test_QP1D12_onlyGhost = O_X_test_QP1D12_onlyGhost
    Y_test_QP1D12 = O_Y_test_QP1D12
    
    X_test_QP1D13 = O_X_test_QP1D13
    X_test_QP1D13_onlyGhost = O_X_test_QP1D13_onlyGhost
    Y_test_QP1D13 = O_Y_test_QP1D13
    
    X_test_QP1D14 = O_X_test_QP1D14
    X_test_QP1D14_onlyGhost = O_X_test_QP1D14_onlyGhost
    Y_test_QP1D14 = O_Y_test_QP1D14
    
    X_test_QP1D15 = O_X_test_QP1D15
    X_test_QP1D15_onlyGhost = O_X_test_QP1D15_onlyGhost
    Y_test_QP1D15 = O_Y_test_QP1D15
    
    X_test_QP1D16 = O_X_test_QP1D16
    X_test_QP1D16_onlyGhost = O_X_test_QP1D16_onlyGhost
    Y_test_QP1D16 = O_Y_test_QP1D16
    
    X_test_QP1D18 = O_X_test_QP1D18
    X_test_QP1D18_onlyGhost = O_X_test_QP1D18_onlyGhost
    Y_test_QP1D18 = O_Y_test_QP1D18
    
    X_test_QP1D19 = O_X_test_QP1D19
    X_test_QP1D19_onlyGhost = O_X_test_QP1D19_onlyGhost
    Y_test_QP1D19 = O_Y_test_QP1D19
    
    X_test_QP1D20 = O_X_test_QP1D20
    X_test_QP1D20_onlyGhost = O_X_test_QP1D20_onlyGhost
    Y_test_QP1D20 = O_Y_test_QP1D20
    
    X_test_QP1D21 = O_X_test_QP1D21
    X_test_QP1D21_onlyGhost = O_X_test_QP1D21_onlyGhost
    Y_test_QP1D21 = O_Y_test_QP1D21
    
    X_test_QP1D22 = O_X_test_QP1D22
    X_test_QP1D22_onlyGhost = O_X_test_QP1D22_onlyGhost
    Y_test_QP1D22 = O_Y_test_QP1D22
    
    X_test_QP1D23 = O_X_test_QP1D23
    X_test_QP1D23_onlyGhost = O_X_test_QP1D23_onlyGhost
    Y_test_QP1D23 = O_Y_test_QP1D23
    
    X_test_QP1D24 = O_X_test_QP1D24
    X_test_QP1D24_onlyGhost = O_X_test_QP1D24_onlyGhost
    Y_test_QP1D24 = O_Y_test_QP1D24
    
    X_test_QP1D25 = O_X_test_QP1D25
    X_test_QP1D25_onlyGhost = O_X_test_QP1D25_onlyGhost
    Y_test_QP1D25 = O_Y_test_QP1D25
    
    X_test_QP1D26 = O_X_test_QP1D26
    X_test_QP1D26_onlyGhost = O_X_test_QP1D26_onlyGhost
    Y_test_QP1D26 = O_Y_test_QP1D26
    
    X_test_QP1D27 = O_X_test_QP1D27
    X_test_QP1D27_onlyGhost = O_X_test_QP1D27_onlyGhost
    Y_test_QP1D27 = O_Y_test_QP1D27
    
    X_test_QP1D29 = O_X_test_QP1D29
    X_test_QP1D29_onlyGhost = O_X_test_QP1D29_onlyGhost
    Y_test_QP1D29 = O_Y_test_QP1D29
    
    X_test_QP1D30 = O_X_test_QP1D30
    X_test_QP1D30_onlyGhost = O_X_test_QP1D30_onlyGhost
    Y_test_QP1D30 = O_Y_test_QP1D30
    
    X_test_QP1D34 = O_X_test_QP1D34
    X_test_QP1D34_onlyGhost = O_X_test_QP1D34_onlyGhost
    Y_test_QP1D34 = O_Y_test_QP1D34
    
    X_test_QP1D35 = O_X_test_QP1D35
    X_test_QP1D35_onlyGhost = O_X_test_QP1D35_onlyGhost
    Y_test_QP1D35 = O_Y_test_QP1D35
    
    X_test_QP1D40 = O_X_test_QP1D40
    X_test_QP1D40_onlyGhost = O_X_test_QP1D40_onlyGhost
    Y_test_QP1D40 = O_Y_test_QP1D40
    
    X_test_QP1D45 = O_X_test_QP1D45
    X_test_QP1D45_onlyGhost = O_X_test_QP1D45_onlyGhost
    Y_test_QP1D45 = O_Y_test_QP1D45
    

    # テストデータ・ラベルの処理（閾値）- 30パターン
    X_QP1D1_test_old, QP1D1_test_QP = O_QP1D1_test_old, O_QP1D1_test_final_QP
    X_QP1D3_test_old, QP1D3_test_QP = O_QP1D3_test_old, O_QP1D3_test_final_QP
    X_QP1D4_test_old, QP1D4_test_QP = O_QP1D4_test_old, O_QP1D4_test_final_QP
    X_QP1D5_test_old, QP1D5_test_QP = O_QP1D5_test_old, O_QP1D5_test_final_QP
    X_QP1D6_test_old, QP1D6_test_QP = O_QP1D6_test_old, O_QP1D6_test_final_QP
    X_QP1D8_test_old, QP1D8_test_QP = O_QP1D8_test_old, O_QP1D8_test_final_QP
    X_QP1D9_test_old, QP1D9_test_QP = O_QP1D9_test_old, O_QP1D9_test_final_QP
    X_QP1D10_test_old, QP1D10_test_QP = O_QP1D10_test_old, O_QP1D10_test_final_QP
    X_QP1D11_test_old, QP1D11_test_QP = O_QP1D11_test_old, O_QP1D11_test_final_QP
    X_QP1D12_test_old, QP1D12_test_QP = O_QP1D12_test_old, O_QP1D12_test_final_QP
    X_QP1D13_test_old, QP1D13_test_QP = O_QP1D13_test_old, O_QP1D13_test_final_QP
    X_QP1D14_test_old, QP1D14_test_QP = O_QP1D14_test_old, O_QP1D14_test_final_QP
    X_QP1D15_test_old, QP1D15_test_QP = O_QP1D15_test_old, O_QP1D15_test_final_QP
    X_QP1D16_test_old, QP1D16_test_QP = O_QP1D16_test_old, O_QP1D16_test_final_QP
    X_QP1D18_test_old, QP1D18_test_QP = O_QP1D18_test_old, O_QP1D18_test_final_QP
    X_QP1D19_test_old, QP1D19_test_QP = O_QP1D19_test_old, O_QP1D19_test_final_QP
    X_QP1D20_test_old, QP1D20_test_QP = O_QP1D20_test_old, O_QP1D20_test_final_QP
    X_QP1D21_test_old, QP1D21_test_QP = O_QP1D21_test_old, O_QP1D21_test_final_QP
    X_QP1D22_test_old, QP1D22_test_QP = O_QP1D22_test_old, O_QP1D22_test_final_QP
    X_QP1D23_test_old, QP1D23_test_QP = O_QP1D23_test_old, O_QP1D23_test_final_QP
    X_QP1D24_test_old, QP1D24_test_QP = O_QP1D24_test_old, O_QP1D24_test_final_QP
    X_QP1D25_test_old, QP1D25_test_QP = O_QP1D25_test_old, O_QP1D25_test_final_QP
    X_QP1D26_test_old, QP1D26_test_QP = O_QP1D26_test_old, O_QP1D26_test_final_QP
    X_QP1D27_test_old, QP1D27_test_QP = O_QP1D27_test_old, O_QP1D27_test_final_QP
    X_QP1D29_test_old, QP1D29_test_QP = O_QP1D29_test_old, O_QP1D29_test_final_QP
    X_QP1D30_test_old, QP1D30_test_QP = O_QP1D30_test_old, O_QP1D30_test_final_QP
    X_QP1D34_test_old, QP1D34_test_QP = O_QP1D34_test_old, O_QP1D34_test_final_QP
    X_QP1D35_test_old, QP1D35_test_QP = O_QP1D35_test_old, O_QP1D35_test_final_QP
    X_QP1D40_test_old, QP1D40_test_QP = O_QP1D40_test_old, O_QP1D40_test_final_QP
    X_QP1D45_test_old, QP1D45_test_QP = O_QP1D45_test_old, O_QP1D45_test_final_QP
    
    

    # 30パターン
    QP1D1_best_threshold = 0
    QP1D1_best_accuracy = 0
    QP1D1_best_predicted_labels = []
    QP1D1_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D1_test_old[i], QP1D1_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D1]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D1_best_accuracy:
            QP1D1_best_accuracy = accuracy
            QP1D1_best_threshold = threshold
            QP1D1_best_predicted_labels = predicted_labels
            QP1D1_best_ground_truth_labels = ground_truth_labels  
    
    
    QP1D3_best_threshold = 0
    QP1D3_best_accuracy = 0
    QP1D3_best_predicted_labels = []
    QP1D3_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D3_test_old[i], QP1D3_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D3]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D3_best_accuracy:
            QP1D3_best_accuracy = accuracy
            QP1D3_best_threshold = threshold
            QP1D3_best_predicted_labels = predicted_labels
            QP1D3_best_ground_truth_labels = ground_truth_labels
            
    QP1D4_best_threshold = 0
    QP1D4_best_accuracy = 0
    QP1D4_best_predicted_labels = []
    QP1D4_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D4_test_old[i], QP1D4_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D4]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D4_best_accuracy:
            QP1D4_best_accuracy = accuracy
            QP1D4_best_threshold = threshold
            QP1D4_best_predicted_labels = predicted_labels
            QP1D4_best_ground_truth_labels = ground_truth_labels
            
    QP1D5_best_threshold = 0
    QP1D5_best_accuracy = 0
    QP1D5_best_predicted_labels = []
    QP1D5_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D5_test_old[i], QP1D5_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D5]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D5_best_accuracy:
            QP1D5_best_accuracy = accuracy
            QP1D5_best_threshold = threshold
            QP1D5_best_predicted_labels = predicted_labels
            QP1D5_best_ground_truth_labels = ground_truth_labels            
            
    QP1D6_best_threshold = 0
    QP1D6_best_accuracy = 0
    QP1D6_best_predicted_labels = []
    QP1D6_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D6_test_old[i], QP1D6_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D6]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D6_best_accuracy:
            QP1D6_best_accuracy = accuracy
            QP1D6_best_threshold = threshold
            QP1D6_best_predicted_labels = predicted_labels
            QP1D6_best_ground_truth_labels = ground_truth_labels           
            
    QP1D8_best_threshold = 0
    QP1D8_best_accuracy = 0
    QP1D8_best_predicted_labels = []
    QP1D8_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D8_test_old[i], QP1D8_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D8]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D8_best_accuracy:
            QP1D8_best_accuracy = accuracy
            QP1D8_best_threshold = threshold
            QP1D8_best_predicted_labels = predicted_labels
            QP1D8_best_ground_truth_labels = ground_truth_labels            
            
    QP1D9_best_threshold = 0
    QP1D9_best_accuracy = 0
    QP1D9_best_predicted_labels = []
    QP1D9_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D9_test_old[i], QP1D9_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D9]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D9_best_accuracy:
            QP1D9_best_accuracy = accuracy
            QP1D9_best_threshold = threshold
            QP1D9_best_predicted_labels = predicted_labels
            QP1D9_best_ground_truth_labels = ground_truth_labels            
            
    QP1D10_best_threshold = 0
    QP1D10_best_accuracy = 0
    QP1D10_best_predicted_labels = []
    QP1D10_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D10_test_old[i], QP1D10_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D10]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D10_best_accuracy:
            QP1D10_best_accuracy = accuracy
            QP1D10_best_threshold = threshold
            QP1D10_best_predicted_labels = predicted_labels
            QP1D10_best_ground_truth_labels = ground_truth_labels
            
    QP1D11_best_threshold = 0
    QP1D11_best_accuracy = 0
    QP1D11_best_predicted_labels = []
    QP1D11_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D11_test_old[i], QP1D11_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D11]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D11_best_accuracy:
            QP1D11_best_accuracy = accuracy
            QP1D11_best_threshold = threshold
            QP1D11_best_predicted_labels = predicted_labels
            QP1D11_best_ground_truth_labels = ground_truth_labels            
                 
    QP1D12_best_threshold = 0
    QP1D12_best_accuracy = 0
    QP1D12_best_predicted_labels = []
    QP1D12_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D12_test_old[i], QP1D12_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D12]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D12_best_accuracy:
            QP1D12_best_accuracy = accuracy
            QP1D12_best_threshold = threshold
            QP1D12_best_predicted_labels = predicted_labels
            QP1D12_best_ground_truth_labels = ground_truth_labels            
            
            
    QP1D13_best_threshold = 0
    QP1D13_best_accuracy = 0
    QP1D13_best_predicted_labels = []
    QP1D13_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D13_test_old[i], QP1D13_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D13]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D13_best_accuracy:
            QP1D13_best_accuracy = accuracy
            QP1D13_best_threshold = threshold
            QP1D13_best_predicted_labels = predicted_labels
            QP1D13_best_ground_truth_labels = ground_truth_labels            
            
            
    QP1D14_best_threshold = 0
    QP1D14_best_accuracy = 0
    QP1D14_best_predicted_labels = []
    QP1D14_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D14_test_old[i], QP1D14_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D14]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D14_best_accuracy:
            QP1D14_best_accuracy = accuracy
            QP1D14_best_threshold = threshold
            QP1D14_best_predicted_labels = predicted_labels
            QP1D14_best_ground_truth_labels = ground_truth_labels           
            
    QP1D15_best_threshold = 0
    QP1D15_best_accuracy = 0
    QP1D15_best_predicted_labels = []
    QP1D15_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D15_test_old[i], QP1D15_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D15]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D15_best_accuracy:
            QP1D15_best_accuracy = accuracy
            QP1D15_best_threshold = threshold
            QP1D15_best_predicted_labels = predicted_labels
            QP1D15_best_ground_truth_labels = ground_truth_labels            
            

    QP1D16_best_threshold = 0
    QP1D16_best_accuracy = 0
    QP1D16_best_predicted_labels = []
    QP1D16_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D16_test_old[i], QP1D16_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D16]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D16_best_accuracy:
            QP1D16_best_accuracy = accuracy
            QP1D16_best_threshold = threshold
            QP1D16_best_predicted_labels = predicted_labels
            QP1D16_best_ground_truth_labels = ground_truth_labels            
            
            
    QP1D18_best_threshold = 0
    QP1D18_best_accuracy = 0
    QP1D18_best_predicted_labels = []
    QP1D18_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D18_test_old[i], QP1D18_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D18]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D18_best_accuracy:
            QP1D18_best_accuracy = accuracy
            QP1D18_best_threshold = threshold
            QP1D18_best_predicted_labels = predicted_labels
            QP1D18_best_ground_truth_labels = ground_truth_labels            
            
    QP1D19_best_threshold = 0
    QP1D19_best_accuracy = 0
    QP1D19_best_predicted_labels = []
    QP1D19_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D19_test_old[i], QP1D19_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D19]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D19_best_accuracy:
            QP1D19_best_accuracy = accuracy
            QP1D19_best_threshold = threshold
            QP1D19_best_predicted_labels = predicted_labels
            QP1D19_best_ground_truth_labels = ground_truth_labels
            
            
    QP1D20_best_threshold = 0
    QP1D20_best_accuracy = 0
    QP1D20_best_predicted_labels = []
    QP1D20_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D20_test_old[i], QP1D20_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D20]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D20_best_accuracy:
            QP1D20_best_accuracy = accuracy
            QP1D20_best_threshold = threshold
            QP1D20_best_predicted_labels = predicted_labels
            QP1D20_best_ground_truth_labels = ground_truth_labels
            
    
    QP1D21_best_threshold = 0
    QP1D21_best_accuracy = 0
    QP1D21_best_predicted_labels = []
    QP1D21_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D21_test_old[i], QP1D21_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D21]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D21_best_accuracy:
            QP1D21_best_accuracy = accuracy
            QP1D21_best_threshold = threshold
            QP1D21_best_predicted_labels = predicted_labels
            QP1D21_best_ground_truth_labels = ground_truth_labels    
    
    
    QP1D22_best_threshold = 0
    QP1D22_best_accuracy = 0
    QP1D22_best_predicted_labels = []
    QP1D22_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D22_test_old[i], QP1D22_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D22]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D22_best_accuracy:
            QP1D22_best_accuracy = accuracy
            QP1D22_best_threshold = threshold
            QP1D22_best_predicted_labels = predicted_labels
            QP1D22_best_ground_truth_labels = ground_truth_labels    
    
    
    QP1D23_best_threshold = 0
    QP1D23_best_accuracy = 0
    QP1D23_best_predicted_labels = []
    QP1D23_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D23_test_old[i], QP1D23_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D23]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D23_best_accuracy:
            QP1D23_best_accuracy = accuracy
            QP1D23_best_threshold = threshold
            QP1D23_best_predicted_labels = predicted_labels
            QP1D23_best_ground_truth_labels = ground_truth_labels    
    
    
    QP1D24_best_threshold = 0
    QP1D24_best_accuracy = 0
    QP1D24_best_predicted_labels = []
    QP1D24_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D24_test_old[i], QP1D24_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D24]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D24_best_accuracy:
            QP1D24_best_accuracy = accuracy
            QP1D24_best_threshold = threshold
            QP1D24_best_predicted_labels = predicted_labels
            QP1D24_best_ground_truth_labels = ground_truth_labels            
            
            
    QP1D25_best_threshold = 0
    QP1D25_best_accuracy = 0
    QP1D25_best_predicted_labels = []
    QP1D25_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D25_test_old[i], QP1D25_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D25]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D25_best_accuracy:
            QP1D25_best_accuracy = accuracy
            QP1D25_best_threshold = threshold
            QP1D25_best_predicted_labels = predicted_labels
            QP1D25_best_ground_truth_labels = ground_truth_labels           
            
    QP1D26_best_threshold = 0
    QP1D26_best_accuracy = 0
    QP1D26_best_predicted_labels = []
    QP1D26_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D26_test_old[i], QP1D26_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D26]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D26_best_accuracy:
            QP1D26_best_accuracy = accuracy
            QP1D26_best_threshold = threshold
            QP1D26_best_predicted_labels = predicted_labels
            QP1D26_best_ground_truth_labels = ground_truth_labels            
            

    QP1D27_best_threshold = 0
    QP1D27_best_accuracy = 0
    QP1D27_best_predicted_labels = []
    QP1D27_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D27_test_old[i], QP1D27_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D27]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D27_best_accuracy:
            QP1D27_best_accuracy = accuracy
            QP1D27_best_threshold = threshold
            QP1D27_best_predicted_labels = predicted_labels
            QP1D27_best_ground_truth_labels = ground_truth_labels            
            
            
    QP1D29_best_threshold = 0
    QP1D29_best_accuracy = 0
    QP1D29_best_predicted_labels = []
    QP1D29_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D29_test_old[i], QP1D29_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D29]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D29_best_accuracy:
            QP1D29_best_accuracy = accuracy
            QP1D29_best_threshold = threshold
            QP1D29_best_predicted_labels = predicted_labels
            QP1D29_best_ground_truth_labels = ground_truth_labels            
            
    QP1D30_best_threshold = 0
    QP1D30_best_accuracy = 0
    QP1D30_best_predicted_labels = []
    QP1D30_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D30_test_old[i], QP1D30_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D30]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D30_best_accuracy:
            QP1D30_best_accuracy = accuracy
            QP1D30_best_threshold = threshold
            QP1D30_best_predicted_labels = predicted_labels
            QP1D30_best_ground_truth_labels = ground_truth_labels
            
            
    QP1D34_best_threshold = 0
    QP1D34_best_accuracy = 0
    QP1D34_best_predicted_labels = []
    QP1D34_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D34_test_old[i], QP1D34_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D34]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D34_best_accuracy:
            QP1D34_best_accuracy = accuracy
            QP1D34_best_threshold = threshold
            QP1D34_best_predicted_labels = predicted_labels
            QP1D34_best_ground_truth_labels = ground_truth_labels
            
    
    QP1D35_best_threshold = 0
    QP1D35_best_accuracy = 0
    QP1D35_best_predicted_labels = []
    QP1D35_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D35_test_old[i], QP1D35_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D35]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D35_best_accuracy:
            QP1D35_best_accuracy = accuracy
            QP1D35_best_threshold = threshold
            QP1D35_best_predicted_labels = predicted_labels
            QP1D35_best_ground_truth_labels = ground_truth_labels    
    
    
    QP1D40_best_threshold = 0
    QP1D40_best_accuracy = 0
    QP1D40_best_predicted_labels = []
    QP1D40_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D40_test_old[i], QP1D40_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D40]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D40_best_accuracy:
            QP1D40_best_accuracy = accuracy
            QP1D40_best_threshold = threshold
            QP1D40_best_predicted_labels = predicted_labels
            QP1D40_best_ground_truth_labels = ground_truth_labels    
    
    
    QP1D45_best_threshold = 0
    QP1D45_best_accuracy = 0
    QP1D45_best_predicted_labels = []
    QP1D45_best_ground_truth_labels = []
    
    for threshold in np.arange(0.01,1.01,0.01):
        test_old = [is_double_compressed(X_QP1D45_test_old[i], QP1D45_test_QP[i], threshold) for i in range(20)]
        predicted_labels = [int(is_double) for is_double in test_old]
        ground_truth_labels = [label for label in Y_test_QP1D45]
        accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
        
        if accuracy > QP1D45_best_accuracy:
            QP1D45_best_accuracy = accuracy
            QP1D45_best_threshold = threshold
            QP1D45_best_predicted_labels = predicted_labels
            QP1D45_best_ground_truth_labels = ground_truth_labels      
    
    
    best_val_score_RBF, best_svm_model_RBF, best_c_value_RBF = 0, None, None
    best_val_score_onlyGhost_RBF, best_svm_model_onlyGhost_RBF, best_c_value_onlyGhost_RBF = 0, None, None

    best_val_score_LINEAR, best_svm_model_LINEAR, best_c_value_LINEAR = 0, None, None
    best_val_score_onlyGhost_LINEAR, best_svm_model_onlyGhost_LINEAR, best_c_value_onlyGhost_LINEAR = 0, None, None
    
    for C_value in C_values['C']:    
        # SVMモデルのインスタンスを作成
        svm_model_RBF = SVC(kernel='rbf', C=C_value)
        svm_model_onlyGhost_RBF = SVC(kernel='rbf', C=C_value)
        
        svm_model_LINEAR = SVC(kernel='linear', C=C_value)
        svm_model_onlyGhost_LINEAR = SVC(kernel='linear', C=C_value)
        

        # 訓練データで訓練
        svm_model_RBF.fit(X_train, Y_train)
        svm_model_onlyGhost_RBF.fit(X_train_onlyGhost, Y_train)
        
        svm_model_LINEAR.fit(X_train, Y_train)
        svm_model_onlyGhost_LINEAR.fit(X_train_onlyGhost, Y_train)
        
        val_accuracy_RBF = accuracy_score(Y_val, svm_model_RBF.predict(X_val))
        val_accuracy_onlyGhost_RBF = accuracy_score(Y_val, svm_model_onlyGhost_RBF.predict(X_val_onlyGhost))
        
        val_accuracy_LINEAR = accuracy_score(Y_val, svm_model_LINEAR.predict(X_val))
        val_accuracy_onlyGhost_LINEAR = accuracy_score(Y_val, svm_model_onlyGhost_LINEAR.predict(X_val_onlyGhost))
                

        # 検証データでの精度が最も高かった場合、そのモデルを保存
        if val_accuracy_RBF > best_val_score_RBF:
            best_val_score_RBF, best_svm_model_RBF, best_c_value_RBF = val_accuracy_RBF, svm_model_RBF, C_value

        if val_accuracy_onlyGhost_RBF > best_val_score_onlyGhost_RBF:
            best_val_score_onlyGhost_RBF, best_svm_model_onlyGhost_RBF, best_c_value_onlyGhost_RBF = val_accuracy_onlyGhost_RBF, svm_model_onlyGhost_RBF, C_value
            
        if val_accuracy_LINEAR > best_val_score_LINEAR:
            best_val_score_LINEAR, best_svm_model_LINEAR, best_c_value_LINEAR = val_accuracy_LINEAR, svm_model_LINEAR, C_value

        if val_accuracy_onlyGhost_LINEAR > best_val_score_onlyGhost_LINEAR:
            best_val_score_onlyGhost_LINEAR, best_svm_model_onlyGhost_LINEAR, best_c_value_onlyGhost_LINEAR = val_accuracy_onlyGhost_LINEAR, svm_model_onlyGhost_LINEAR, C_value
            
    # テストデータで評価 - 30　パターン
    QP1D1_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D1)
    QP1D1_accuracy_RBF = accuracy_score(Y_test_QP1D1, QP1D1_predictions_RBF)
    
    QP1D1_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D1)
    QP1D1_accuracy_LINEAR = accuracy_score(Y_test_QP1D1, QP1D1_predictions_LINEAR)
    
    QP1D1_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D1_onlyGhost)
    QP1D1_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D1, QP1D1_predictions_onlyGhost_RBF)
    
    QP1D1_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D1_onlyGhost)
    QP1D1_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D1, QP1D1_predictions_onlyGhost_LINEAR)
    
    
    QP1D3_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D3)
    QP1D3_accuracy_RBF = accuracy_score(Y_test_QP1D3, QP1D3_predictions_RBF)
    
    QP1D3_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D3)
    QP1D3_accuracy_LINEAR = accuracy_score(Y_test_QP1D3, QP1D3_predictions_LINEAR)
    
    QP1D3_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D3_onlyGhost)
    QP1D3_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D3, QP1D3_predictions_onlyGhost_RBF)
    
    QP1D3_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D3_onlyGhost)
    QP1D3_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D3, QP1D3_predictions_onlyGhost_LINEAR)
    
    
    QP1D4_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D4)
    QP1D4_accuracy_RBF = accuracy_score(Y_test_QP1D4, QP1D4_predictions_RBF)
    
    QP1D4_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D4)
    QP1D4_accuracy_LINEAR = accuracy_score(Y_test_QP1D4, QP1D4_predictions_LINEAR)
    
    QP1D4_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D4_onlyGhost)
    QP1D4_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D4, QP1D4_predictions_onlyGhost_RBF)
    
    QP1D4_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D4_onlyGhost)
    QP1D4_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D4, QP1D4_predictions_onlyGhost_LINEAR)
    

    QP1D5_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D5)
    QP1D5_accuracy_RBF = accuracy_score(Y_test_QP1D5, QP1D5_predictions_RBF)
    
    QP1D5_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D5)
    QP1D5_accuracy_LINEAR = accuracy_score(Y_test_QP1D5, QP1D5_predictions_LINEAR)
    
    QP1D5_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D5_onlyGhost)
    QP1D5_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D5, QP1D5_predictions_onlyGhost_RBF)
    
    QP1D5_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D5_onlyGhost)
    QP1D5_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D5, QP1D5_predictions_onlyGhost_LINEAR)
    
    
    QP1D6_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D6)
    QP1D6_accuracy_RBF = accuracy_score(Y_test_QP1D6, QP1D6_predictions_RBF)
    
    QP1D6_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D6)
    QP1D6_accuracy_LINEAR = accuracy_score(Y_test_QP1D6, QP1D6_predictions_LINEAR)
    
    QP1D6_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D6_onlyGhost)
    QP1D6_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D6, QP1D6_predictions_onlyGhost_RBF)
    
    QP1D6_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D6_onlyGhost)
    QP1D6_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D6, QP1D6_predictions_onlyGhost_LINEAR)
    

    QP1D8_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D8)
    QP1D8_accuracy_RBF = accuracy_score(Y_test_QP1D8, QP1D8_predictions_RBF)
    
    QP1D8_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D8)
    QP1D8_accuracy_LINEAR = accuracy_score(Y_test_QP1D8, QP1D8_predictions_LINEAR)
    
    QP1D8_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D8_onlyGhost)
    QP1D8_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D8, QP1D8_predictions_onlyGhost_RBF)
    
    QP1D8_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D8_onlyGhost)
    QP1D8_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D8, QP1D8_predictions_onlyGhost_LINEAR)
    
    
    QP1D9_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D9)
    QP1D9_accuracy_RBF = accuracy_score(Y_test_QP1D9, QP1D9_predictions_RBF)
    
    QP1D9_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D9)
    QP1D9_accuracy_LINEAR = accuracy_score(Y_test_QP1D9, QP1D9_predictions_LINEAR)
    
    QP1D9_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D9_onlyGhost)
    QP1D9_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D9, QP1D9_predictions_onlyGhost_RBF)
    
    QP1D9_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D9_onlyGhost)
    QP1D9_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D9, QP1D9_predictions_onlyGhost_LINEAR)
    
    
    QP1D10_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D10)
    QP1D10_accuracy_RBF = accuracy_score(Y_test_QP1D10, QP1D10_predictions_RBF)
    
    QP1D10_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D10)
    QP1D10_accuracy_LINEAR = accuracy_score(Y_test_QP1D10, QP1D10_predictions_LINEAR)
    
    QP1D10_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D10_onlyGhost)
    QP1D10_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D10, QP1D10_predictions_onlyGhost_RBF)
    
    QP1D10_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D10_onlyGhost)
    QP1D10_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D10, QP1D10_predictions_onlyGhost_LINEAR)
    
    
    QP1D11_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D11)
    QP1D11_accuracy_RBF = accuracy_score(Y_test_QP1D11, QP1D11_predictions_RBF)
    
    QP1D11_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D11)
    QP1D11_accuracy_LINEAR = accuracy_score(Y_test_QP1D11, QP1D11_predictions_LINEAR)
    
    QP1D11_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D11_onlyGhost)
    QP1D11_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D11, QP1D11_predictions_onlyGhost_RBF)
    
    QP1D11_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D11_onlyGhost)
    QP1D11_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D11, QP1D11_predictions_onlyGhost_LINEAR)
    

    QP1D12_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D12)
    QP1D12_accuracy_RBF = accuracy_score(Y_test_QP1D12, QP1D12_predictions_RBF)
    
    QP1D12_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D12)
    QP1D12_accuracy_LINEAR = accuracy_score(Y_test_QP1D12, QP1D12_predictions_LINEAR)
    
    QP1D12_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D12_onlyGhost)
    QP1D12_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D12, QP1D12_predictions_onlyGhost_RBF)
    
    QP1D12_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D12_onlyGhost)
    QP1D12_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D12, QP1D12_predictions_onlyGhost_LINEAR)
    
    
    QP1D13_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D13)
    QP1D13_accuracy_RBF = accuracy_score(Y_test_QP1D13, QP1D13_predictions_RBF)
    
    QP1D13_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D13)
    QP1D13_accuracy_LINEAR = accuracy_score(Y_test_QP1D13, QP1D13_predictions_LINEAR)
    
    QP1D13_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D13_onlyGhost)
    QP1D13_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D13, QP1D13_predictions_onlyGhost_RBF)
    
    QP1D13_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D13_onlyGhost)
    QP1D13_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D13, QP1D13_predictions_onlyGhost_LINEAR)
    
    
    QP1D14_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D14)
    QP1D14_accuracy_RBF = accuracy_score(Y_test_QP1D14, QP1D14_predictions_RBF)
    
    QP1D14_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D14)
    QP1D14_accuracy_LINEAR = accuracy_score(Y_test_QP1D14, QP1D14_predictions_LINEAR)
    
    QP1D14_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D14_onlyGhost)
    QP1D14_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D14, QP1D14_predictions_onlyGhost_RBF)
    
    QP1D14_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D14_onlyGhost)
    QP1D14_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D14, QP1D14_predictions_onlyGhost_LINEAR)
    
    
    QP1D15_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D15)
    QP1D15_accuracy_RBF = accuracy_score(Y_test_QP1D15, QP1D15_predictions_RBF)
    
    QP1D15_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D15)
    QP1D15_accuracy_LINEAR = accuracy_score(Y_test_QP1D15, QP1D15_predictions_LINEAR)
    
    QP1D15_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D15_onlyGhost)
    QP1D15_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D15, QP1D15_predictions_onlyGhost_RBF)
    
    QP1D15_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D15_onlyGhost)
    QP1D15_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D15, QP1D15_predictions_onlyGhost_LINEAR)
    

    QP1D16_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D16)
    QP1D16_accuracy_RBF = accuracy_score(Y_test_QP1D16, QP1D16_predictions_RBF)
    
    QP1D16_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D16)
    QP1D16_accuracy_LINEAR = accuracy_score(Y_test_QP1D16, QP1D16_predictions_LINEAR)
    
    QP1D16_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D16_onlyGhost)
    QP1D16_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D16, QP1D16_predictions_onlyGhost_RBF)
    
    QP1D16_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D16_onlyGhost)
    QP1D16_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D16, QP1D16_predictions_onlyGhost_LINEAR)
    
    
    QP1D18_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D18)
    QP1D18_accuracy_RBF = accuracy_score(Y_test_QP1D18, QP1D18_predictions_RBF)
    
    QP1D18_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D18)
    QP1D18_accuracy_LINEAR = accuracy_score(Y_test_QP1D18, QP1D18_predictions_LINEAR)
    
    QP1D18_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D18_onlyGhost)
    QP1D18_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D18, QP1D18_predictions_onlyGhost_RBF)
    
    QP1D18_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D18_onlyGhost)
    QP1D18_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D18, QP1D18_predictions_onlyGhost_LINEAR)
    

    QP1D19_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D19)
    QP1D19_accuracy_RBF = accuracy_score(Y_test_QP1D19, QP1D19_predictions_RBF)
    
    QP1D19_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D19)
    QP1D19_accuracy_LINEAR = accuracy_score(Y_test_QP1D19, QP1D19_predictions_LINEAR)
    
    QP1D19_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D19_onlyGhost)
    QP1D19_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D19, QP1D19_predictions_onlyGhost_RBF)
    
    QP1D19_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D19_onlyGhost)
    QP1D19_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D19, QP1D19_predictions_onlyGhost_LINEAR)
    
    
    QP1D20_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D20)
    QP1D20_accuracy_RBF = accuracy_score(Y_test_QP1D20, QP1D20_predictions_RBF)
    
    QP1D20_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D20)
    QP1D20_accuracy_LINEAR = accuracy_score(Y_test_QP1D20, QP1D20_predictions_LINEAR)
    
    QP1D20_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D20_onlyGhost)
    QP1D20_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D20, QP1D20_predictions_onlyGhost_RBF)
    
    QP1D20_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D20_onlyGhost)
    QP1D20_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D20, QP1D20_predictions_onlyGhost_LINEAR)
    
    
    QP1D21_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D21)
    QP1D21_accuracy_RBF = accuracy_score(Y_test_QP1D21, QP1D21_predictions_RBF)
    
    QP1D21_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D21)
    QP1D21_accuracy_LINEAR = accuracy_score(Y_test_QP1D21, QP1D21_predictions_LINEAR)
    
    QP1D21_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D21_onlyGhost)
    QP1D21_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D21, QP1D21_predictions_onlyGhost_RBF)
    
    QP1D21_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D21_onlyGhost)
    QP1D21_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D21, QP1D21_predictions_onlyGhost_LINEAR)
    
    
    QP1D22_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D22)
    QP1D22_accuracy_RBF = accuracy_score(Y_test_QP1D22, QP1D22_predictions_RBF)
    
    QP1D22_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D22)
    QP1D22_accuracy_LINEAR = accuracy_score(Y_test_QP1D22, QP1D22_predictions_LINEAR)
    
    QP1D22_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D22_onlyGhost)
    QP1D22_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D22, QP1D22_predictions_onlyGhost_RBF)
    
    QP1D22_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D22_onlyGhost)
    QP1D22_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D22, QP1D22_predictions_onlyGhost_LINEAR)
    

    QP1D23_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D23)
    QP1D23_accuracy_RBF = accuracy_score(Y_test_QP1D23, QP1D23_predictions_RBF)
    
    QP1D23_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D23)
    QP1D23_accuracy_LINEAR = accuracy_score(Y_test_QP1D23, QP1D23_predictions_LINEAR)
    
    QP1D23_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D23_onlyGhost)
    QP1D23_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D23, QP1D23_predictions_onlyGhost_RBF)
    
    QP1D23_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D23_onlyGhost)
    QP1D23_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D23, QP1D23_predictions_onlyGhost_LINEAR)    
    
    
    QP1D24_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D24)
    QP1D24_accuracy_RBF = accuracy_score(Y_test_QP1D24, QP1D24_predictions_RBF)
    
    QP1D24_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D24)
    QP1D24_accuracy_LINEAR = accuracy_score(Y_test_QP1D24, QP1D24_predictions_LINEAR)
    
    QP1D24_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D24_onlyGhost)
    QP1D24_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D24, QP1D24_predictions_onlyGhost_RBF)
    
    QP1D24_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D24_onlyGhost)
    QP1D24_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D24, QP1D24_predictions_onlyGhost_LINEAR)
    
    
    QP1D25_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D25)
    QP1D25_accuracy_RBF = accuracy_score(Y_test_QP1D25, QP1D25_predictions_RBF)
    
    QP1D25_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D25)
    QP1D25_accuracy_LINEAR = accuracy_score(Y_test_QP1D25, QP1D25_predictions_LINEAR)
    
    QP1D25_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D25_onlyGhost)
    QP1D25_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D25, QP1D25_predictions_onlyGhost_RBF)
    
    QP1D25_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D25_onlyGhost)
    QP1D25_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D25, QP1D25_predictions_onlyGhost_LINEAR)
    
    
    QP1D26_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D26)
    QP1D26_accuracy_RBF = accuracy_score(Y_test_QP1D26, QP1D26_predictions_RBF)
    
    QP1D26_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D26)
    QP1D26_accuracy_LINEAR = accuracy_score(Y_test_QP1D26, QP1D26_predictions_LINEAR)
    
    QP1D26_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D26_onlyGhost)
    QP1D26_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D26, QP1D26_predictions_onlyGhost_RBF)
    
    QP1D26_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D26_onlyGhost)
    QP1D26_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D26, QP1D26_predictions_onlyGhost_LINEAR)
    

    QP1D27_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D27)
    QP1D27_accuracy_RBF = accuracy_score(Y_test_QP1D27, QP1D27_predictions_RBF)
    
    QP1D27_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D27)
    QP1D27_accuracy_LINEAR = accuracy_score(Y_test_QP1D27, QP1D27_predictions_LINEAR)
    
    QP1D27_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D27_onlyGhost)
    QP1D27_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D27, QP1D27_predictions_onlyGhost_RBF)
    
    QP1D27_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D27_onlyGhost)
    QP1D27_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D27, QP1D27_predictions_onlyGhost_LINEAR)
    
    
    QP1D29_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D29)
    QP1D29_accuracy_RBF = accuracy_score(Y_test_QP1D29, QP1D29_predictions_RBF)
    
    QP1D29_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D29)
    QP1D29_accuracy_LINEAR = accuracy_score(Y_test_QP1D29, QP1D29_predictions_LINEAR)
    
    QP1D29_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D29_onlyGhost)
    QP1D29_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D29, QP1D29_predictions_onlyGhost_RBF)
    
    QP1D29_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D29_onlyGhost)
    QP1D29_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D29, QP1D29_predictions_onlyGhost_LINEAR)
    

    QP1D30_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D30)
    QP1D30_accuracy_RBF = accuracy_score(Y_test_QP1D30, QP1D30_predictions_RBF)
    
    QP1D30_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D30)
    QP1D30_accuracy_LINEAR = accuracy_score(Y_test_QP1D30, QP1D30_predictions_LINEAR)
    
    QP1D30_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D30_onlyGhost)
    QP1D30_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D30, QP1D30_predictions_onlyGhost_RBF)
    
    QP1D30_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D30_onlyGhost)
    QP1D30_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D30, QP1D30_predictions_onlyGhost_LINEAR)
    
    
    QP1D34_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D34)
    QP1D34_accuracy_RBF = accuracy_score(Y_test_QP1D34, QP1D34_predictions_RBF)
    
    QP1D34_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D34)
    QP1D34_accuracy_LINEAR = accuracy_score(Y_test_QP1D34, QP1D34_predictions_LINEAR)
    
    QP1D34_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D34_onlyGhost)
    QP1D34_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D34, QP1D34_predictions_onlyGhost_RBF)
    
    QP1D34_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D34_onlyGhost)
    QP1D34_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D34, QP1D34_predictions_onlyGhost_LINEAR)
    
    
    QP1D35_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D35)
    QP1D35_accuracy_RBF = accuracy_score(Y_test_QP1D35, QP1D35_predictions_RBF)
    
    QP1D35_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D35)
    QP1D35_accuracy_LINEAR = accuracy_score(Y_test_QP1D35, QP1D35_predictions_LINEAR)
    
    QP1D35_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D35_onlyGhost)
    QP1D35_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D35, QP1D35_predictions_onlyGhost_RBF)
    
    QP1D35_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D35_onlyGhost)
    QP1D35_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D35, QP1D35_predictions_onlyGhost_LINEAR)
    
    
    QP1D40_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D40)
    QP1D40_accuracy_RBF = accuracy_score(Y_test_QP1D40, QP1D40_predictions_RBF)
    
    QP1D40_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D40)
    QP1D40_accuracy_LINEAR = accuracy_score(Y_test_QP1D40, QP1D40_predictions_LINEAR)
    
    QP1D40_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D40_onlyGhost)
    QP1D40_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D40, QP1D40_predictions_onlyGhost_RBF)
    
    QP1D40_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D40_onlyGhost)
    QP1D40_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D40, QP1D40_predictions_onlyGhost_LINEAR)
    

    QP1D45_predictions_RBF = best_svm_model_RBF.predict(X_test_QP1D45)
    QP1D45_accuracy_RBF = accuracy_score(Y_test_QP1D45, QP1D45_predictions_RBF)
    
    QP1D45_predictions_LINEAR = best_svm_model_LINEAR.predict(X_test_QP1D45)
    QP1D45_accuracy_LINEAR = accuracy_score(Y_test_QP1D45, QP1D45_predictions_LINEAR)
    
    QP1D45_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(X_test_QP1D45_onlyGhost)
    QP1D45_accuracy_onlyGhost_RBF = accuracy_score(Y_test_QP1D45, QP1D45_predictions_onlyGhost_RBF)
    
    QP1D45_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(X_test_QP1D45_onlyGhost)
    QP1D45_accuracy_onlyGhost_LINEAR = accuracy_score(Y_test_QP1D45, QP1D45_predictions_onlyGhost_LINEAR)    
    

        
    # Test結果を保存    
    result_row ={'RBF_QP1D1':QP1D1_accuracy_RBF,'LINEAR_QP1D1':QP1D1_accuracy_LINEAR,'RBF_ghost_QP1D1':QP1D1_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D1':QP1D1_accuracy_onlyGhost_LINEAR,'QP1D1_old':QP1D1_best_accuracy,
                 'RBF_QP1D3':QP1D3_accuracy_RBF,'LINEAR_QP1D3':QP1D3_accuracy_LINEAR,'RBF_ghost_QP1D3':QP1D3_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D3':QP1D3_accuracy_onlyGhost_LINEAR,'QP1D3_old':QP1D3_best_accuracy,
                 'RBF_QP1D4':QP1D4_accuracy_RBF,'LINEAR_QP1D4':QP1D4_accuracy_LINEAR,'RBF_ghost_QP1D4':QP1D4_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D4':QP1D4_accuracy_onlyGhost_LINEAR,'QP1D4_old':QP1D4_best_accuracy,
                 'RBF_QP1D5':QP1D5_accuracy_RBF,'LINEAR_QP1D5':QP1D5_accuracy_LINEAR,'RBF_ghost_QP1D5':QP1D5_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D5':QP1D5_accuracy_onlyGhost_LINEAR,'QP1D5_old':QP1D5_best_accuracy,
                 'RBF_QP1D6':QP1D6_accuracy_RBF,'LINEAR_QP1D6':QP1D6_accuracy_LINEAR,'RBF_ghost_QP1D6':QP1D6_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D6':QP1D6_accuracy_onlyGhost_LINEAR,'QP1D6_old':QP1D6_best_accuracy,
                 'RBF_QP1D8':QP1D8_accuracy_RBF,'LINEAR_QP1D8':QP1D8_accuracy_LINEAR,'RBF_ghost_QP1D8':QP1D8_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D8':QP1D8_accuracy_onlyGhost_LINEAR,'QP1D8_old':QP1D8_best_accuracy,
                 'RBF_QP1D9':QP1D9_accuracy_RBF,'LINEAR_QP1D9':QP1D9_accuracy_LINEAR,'RBF_ghost_QP1D9':QP1D9_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D9':QP1D9_accuracy_onlyGhost_LINEAR,'QP1D9_old':QP1D9_best_accuracy,
                 'RBF_QP1D10':QP1D10_accuracy_RBF,'LINEAR_QP1D10':QP1D10_accuracy_LINEAR,'RBF_ghost_QP1D10':QP1D10_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D10':QP1D10_accuracy_onlyGhost_LINEAR,'QP1D10_old':QP1D10_best_accuracy,
                 'RBF_QP1D11':QP1D11_accuracy_RBF,'LINEAR_QP1D11':QP1D11_accuracy_LINEAR,'RBF_ghost_QP1D11':QP1D11_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D11':QP1D11_accuracy_onlyGhost_LINEAR,'QP1D11_old':QP1D11_best_accuracy,
                 'RBF_QP1D12':QP1D12_accuracy_RBF,'LINEAR_QP1D12':QP1D12_accuracy_LINEAR,'RBF_ghost_QP1D12':QP1D12_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D12':QP1D12_accuracy_onlyGhost_LINEAR,'QP1D12_old':QP1D12_best_accuracy,
                 'RBF_QP1D13':QP1D13_accuracy_RBF,'LINEAR_QP1D13':QP1D13_accuracy_LINEAR,'RBF_ghost_QP1D13':QP1D13_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D13':QP1D13_accuracy_onlyGhost_LINEAR,'QP1D13_old':QP1D13_best_accuracy,
                 'RBF_QP1D14':QP1D14_accuracy_RBF,'LINEAR_QP1D14':QP1D14_accuracy_LINEAR,'RBF_ghost_QP1D14':QP1D14_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D14':QP1D14_accuracy_onlyGhost_LINEAR,'QP1D14_old':QP1D14_best_accuracy,
                 'RBF_QP1D15':QP1D15_accuracy_RBF,'LINEAR_QP1D15':QP1D15_accuracy_LINEAR,'RBF_ghost_QP1D15':QP1D15_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D15':QP1D15_accuracy_onlyGhost_LINEAR,'QP1D15_old':QP1D15_best_accuracy,
                 'RBF_QP1D16':QP1D16_accuracy_RBF,'LINEAR_QP1D16':QP1D16_accuracy_LINEAR,'RBF_ghost_QP1D16':QP1D16_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D16':QP1D16_accuracy_onlyGhost_LINEAR,'QP1D16_old':QP1D16_best_accuracy,
                 'RBF_QP1D18':QP1D18_accuracy_RBF,'LINEAR_QP1D18':QP1D18_accuracy_LINEAR,'RBF_ghost_QP1D18':QP1D18_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D18':QP1D18_accuracy_onlyGhost_LINEAR,'QP1D18_old':QP1D18_best_accuracy,
                 'RBF_QP1D19':QP1D19_accuracy_RBF,'LINEAR_QP1D19':QP1D19_accuracy_LINEAR,'RBF_ghost_QP1D19':QP1D19_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D19':QP1D19_accuracy_onlyGhost_LINEAR,'QP1D19_old':QP1D19_best_accuracy,
                 'RBF_QP1D20':QP1D20_accuracy_RBF,'LINEAR_QP1D20':QP1D20_accuracy_LINEAR,'RBF_ghost_QP1D20':QP1D20_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D20':QP1D20_accuracy_onlyGhost_LINEAR,'QP1D20_old':QP1D20_best_accuracy,
                 'RBF_QP1D21':QP1D21_accuracy_RBF,'LINEAR_QP1D21':QP1D21_accuracy_LINEAR,'RBF_ghost_QP1D21':QP1D21_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D21':QP1D21_accuracy_onlyGhost_LINEAR,'QP1D21_old':QP1D21_best_accuracy,
                 'RBF_QP1D22':QP1D22_accuracy_RBF,'LINEAR_QP1D22':QP1D22_accuracy_LINEAR,'RBF_ghost_QP1D22':QP1D22_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D22':QP1D22_accuracy_onlyGhost_LINEAR,'QP1D22_old':QP1D22_best_accuracy,
                 'RBF_QP1D23':QP1D23_accuracy_RBF,'LINEAR_QP1D23':QP1D23_accuracy_LINEAR,'RBF_ghost_QP1D23':QP1D23_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D23':QP1D23_accuracy_onlyGhost_LINEAR,'QP1D23_old':QP1D23_best_accuracy,
                 'RBF_QP1D24':QP1D24_accuracy_RBF,'LINEAR_QP1D24':QP1D24_accuracy_LINEAR,'RBF_ghost_QP1D24':QP1D24_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D24':QP1D24_accuracy_onlyGhost_LINEAR,'QP1D24_old':QP1D24_best_accuracy,
                 'RBF_QP1D25':QP1D25_accuracy_RBF,'LINEAR_QP1D25':QP1D25_accuracy_LINEAR,'RBF_ghost_QP1D25':QP1D25_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D25':QP1D25_accuracy_onlyGhost_LINEAR,'QP1D25_old':QP1D25_best_accuracy,
                 'RBF_QP1D26':QP1D26_accuracy_RBF,'LINEAR_QP1D26':QP1D26_accuracy_LINEAR,'RBF_ghost_QP1D26':QP1D26_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D26':QP1D26_accuracy_onlyGhost_LINEAR,'QP1D26_old':QP1D26_best_accuracy,
                 'RBF_QP1D27':QP1D27_accuracy_RBF,'LINEAR_QP1D27':QP1D27_accuracy_LINEAR,'RBF_ghost_QP1D27':QP1D27_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D27':QP1D27_accuracy_onlyGhost_LINEAR,'QP1D27_old':QP1D27_best_accuracy,
                 'RBF_QP1D29':QP1D29_accuracy_RBF,'LINEAR_QP1D29':QP1D29_accuracy_LINEAR,'RBF_ghost_QP1D29':QP1D29_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D29':QP1D29_accuracy_onlyGhost_LINEAR,'QP1D29_old':QP1D29_best_accuracy,
                 'RBF_QP1D30':QP1D30_accuracy_RBF,'LINEAR_QP1D30':QP1D30_accuracy_LINEAR,'RBF_ghost_QP1D30':QP1D30_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D30':QP1D30_accuracy_onlyGhost_LINEAR,'QP1D30_old':QP1D30_best_accuracy,
                 'RBF_QP1D34':QP1D34_accuracy_RBF,'LINEAR_QP1D34':QP1D34_accuracy_LINEAR,'RBF_ghost_QP1D34':QP1D34_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D34':QP1D34_accuracy_onlyGhost_LINEAR,'QP1D34_old':QP1D34_best_accuracy,
                 'RBF_QP1D35':QP1D35_accuracy_RBF,'LINEAR_QP1D35':QP1D35_accuracy_LINEAR,'RBF_ghost_QP1D35':QP1D35_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D35':QP1D35_accuracy_onlyGhost_LINEAR,'QP1D35_old':QP1D35_best_accuracy,
                 'RBF_QP1D40':QP1D40_accuracy_RBF,'LINEAR_QP1D40':QP1D40_accuracy_LINEAR,'RBF_ghost_QP1D40':QP1D40_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D40':QP1D40_accuracy_onlyGhost_LINEAR,'QP1D40_old':QP1D40_best_accuracy,
                 'RBF_QP1D45':QP1D45_accuracy_RBF,'LINEAR_QP1D45':QP1D45_accuracy_LINEAR,'RBF_ghost_QP1D45':QP1D45_accuracy_onlyGhost_RBF,'LINEAR_ghost_QP1D45':QP1D45_accuracy_onlyGhost_LINEAR,'QP1D45_old':QP1D45_best_accuracy}

    results = pd.concat([results, pd.DataFrame([result_row])], ignore_index=True)
    print(results)


<Fold-1>

4968 552
  RBF_QP1D1 LINEAR_QP1D1 RBF_ghost_QP1D1 LINEAR_ghost_QP1D1 QP1D1_old RBF_QP1D3 LINEAR_QP1D3 RBF_ghost_QP1D3 LINEAR_ghost_QP1D3 QP1D3_old RBF_QP1D4 LINEAR_QP1D4 RBF_ghost_QP1D4 LINEAR_ghost_QP1D4 QP1D4_old RBF_QP1D5 LINEAR_QP1D5 RBF_ghost_QP1D5 LINEAR_ghost_QP1D5 QP1D5_old RBF_QP1D6 LINEAR_QP1D6 RBF_ghost_QP1D6 LINEAR_ghost_QP1D6 QP1D6_old RBF_QP1D8 LINEAR_QP1D8 RBF_ghost_QP1D8 LINEAR_ghost_QP1D8 QP1D8_old RBF_QP1D9 LINEAR_QP1D9 RBF_ghost_QP1D9 LINEAR_ghost_QP1D9 QP1D9_old RBF_QP1D10 LINEAR_QP1D10 RBF_ghost_QP1D10 LINEAR_ghost_QP1D10 QP1D10_old RBF_QP1D11 LINEAR_QP1D11 RBF_ghost_QP1D11 LINEAR_ghost_QP1D11 QP1D11_old RBF_QP1D12 LINEAR_QP1D12 RBF_ghost_QP1D12 LINEAR_ghost_QP1D12 QP1D12_old RBF_QP1D13 LINEAR_QP1D13 RBF_ghost_QP1D13 LINEAR_ghost_QP1D13 QP1D13_old RBF_QP1D14 LINEAR_QP1D14 RBF_ghost_QP1D14 LINEAR_ghost_QP1D14 QP1D14_old RBF_QP1D15 LINEAR_QP1D15 RBF_ghost_QP1D15 LINEAR_ghost_QP1D15 QP1D15_old RBF_QP1D16 LINEAR_QP1D16 RBF_ghost_QP1D16 LINEAR_ghost_QP1D16 QP1

In [47]:
def print_stats(column_name, label):
    average = round(results[column_name].mean(), 4)
    std_dev = round(results[column_name].std(), 4)
    max_value = round(results[column_name].max(), 4)
    min_value = round(results[column_name].min(), 4)

    print(f'Average Test Score {column_name} {label}: {average}')
    print(f'Standard Deviation of Test Score {column_name} {label}: {std_dev}')
    print(f'Maximum Test Score {column_name} {label}: {max_value}')
    print(f'Minimum Test Score {column_name} {label}: {min_value}')
    print('------------------------------------------------------')

# 'Test_Score'列に関して統計情報を表示
print_stats('RBF_QP1D1', 'with RBF')
print_stats('LINEAR_QP1D1', 'with LINEAR')
print_stats('RBF_ghost_QP1D1', 'with RBF')
print_stats('LINEAR_ghost_QP1D1', 'with LINEAR')
print_stats('QP1D1_old', 'with old model')
print()
print_stats('RBF_QP1D3', 'with RBF')
print_stats('LINEAR_QP1D3', 'with LINEAR')
print_stats('RBF_ghost_QP1D3', 'with RBF')
print_stats('LINEAR_ghost_QP1D3', 'with LINEAR')
print_stats('QP1D3_old', 'with old model')
print()
print_stats('RBF_QP1D4', 'with RBF')
print_stats('LINEAR_QP1D4', 'with LINEAR')
print_stats('RBF_ghost_QP1D4', 'with RBF')
print_stats('LINEAR_ghost_QP1D4', 'with LINEAR')
print_stats('QP1D4_old', 'with old model')
print()
print_stats('RBF_QP1D5', 'with RBF')
print_stats('LINEAR_QP1D5', 'with LINEAR')
print_stats('RBF_ghost_QP1D5', 'with RBF')
print_stats('LINEAR_ghost_QP1D5', 'with LINEAR')
print_stats('QP1D5_old', 'with old model')
print()
print_stats('RBF_QP1D6', 'with RBF')
print_stats('LINEAR_QP1D6', 'with LINEAR')
print_stats('RBF_ghost_QP1D6', 'with RBF')
print_stats('LINEAR_ghost_QP1D6', 'with LINEAR')
print_stats('QP1D6_old', 'with old model')
print()
print_stats('RBF_QP1D8', 'with RBF')
print_stats('LINEAR_QP1D8', 'with LINEAR')
print_stats('RBF_ghost_QP1D8', 'with RBF')
print_stats('LINEAR_ghost_QP1D8', 'with LINEAR')
print_stats('QP1D8_old', 'with old model')
print()
print_stats('RBF_QP1D9', 'with RBF')
print_stats('LINEAR_QP1D9', 'with LINEAR')
print_stats('RBF_ghost_QP1D9', 'with RBF')
print_stats('LINEAR_ghost_QP1D9', 'with LINEAR')
print_stats('QP1D9_old', 'with old model')
print()
print_stats('RBF_QP1D10', 'with RBF')
print_stats('LINEAR_QP1D10', 'with LINEAR')
print_stats('RBF_ghost_QP1D10', 'with RBF')
print_stats('LINEAR_ghost_QP1D10', 'with LINEAR')
print_stats('QP1D10_old', 'with old model')
print()
print_stats('RBF_QP1D11', 'with RBF')
print_stats('LINEAR_QP1D11', 'with LINEAR')
print_stats('RBF_ghost_QP1D11', 'with RBF')
print_stats('LINEAR_ghost_QP1D11', 'with LINEAR')
print_stats('QP1D11_old', 'with old model')
print()
print_stats('RBF_QP1D12', 'with RBF')
print_stats('LINEAR_QP1D12', 'with LINEAR')
print_stats('RBF_ghost_QP1D12', 'with RBF')
print_stats('LINEAR_ghost_QP1D12', 'with LINEAR')
print_stats('QP1D12_old', 'with old model')
print()
print_stats('RBF_QP1D13', 'with RBF')
print_stats('LINEAR_QP1D13', 'with LINEAR')
print_stats('RBF_ghost_QP1D13', 'with RBF')
print_stats('LINEAR_ghost_QP1D13', 'with LINEAR')
print_stats('QP1D13_old', 'with old model')
print()
print_stats('RBF_QP1D14', 'with RBF')
print_stats('LINEAR_QP1D14', 'with LINEAR')
print_stats('RBF_ghost_QP1D14', 'with RBF')
print_stats('LINEAR_ghost_QP1D14', 'with LINEAR')
print_stats('QP1D14_old', 'with old model')
print()
print_stats('RBF_QP1D15', 'with RBF')
print_stats('LINEAR_QP1D15', 'with LINEAR')
print_stats('RBF_ghost_QP1D15', 'with RBF')
print_stats('LINEAR_ghost_QP1D15', 'with LINEAR')
print_stats('QP1D15_old', 'with old model')
print()
print_stats('RBF_QP1D16', 'with RBF')
print_stats('LINEAR_QP1D16', 'with LINEAR')
print_stats('RBF_ghost_QP1D16', 'with RBF')
print_stats('LINEAR_ghost_QP1D16', 'with LINEAR')
print_stats('QP1D16_old', 'with old model')
print()
print_stats('RBF_QP1D18', 'with RBF')
print_stats('LINEAR_QP1D18', 'with LINEAR')
print_stats('RBF_ghost_QP1D18', 'with RBF')
print_stats('LINEAR_ghost_QP1D18', 'with LINEAR')
print_stats('QP1D18_old', 'with old model')
print()
print_stats('RBF_QP1D19', 'with RBF')
print_stats('LINEAR_QP1D19', 'with LINEAR')
print_stats('RBF_ghost_QP1D19', 'with RBF')
print_stats('LINEAR_ghost_QP1D19', 'with LINEAR')
print_stats('QP1D19_old', 'with old model')
print()
print_stats('RBF_QP1D20', 'with RBF')
print_stats('LINEAR_QP1D20', 'with LINEAR')
print_stats('RBF_ghost_QP1D20', 'with RBF')
print_stats('LINEAR_ghost_QP1D20', 'with LINEAR')
print_stats('QP1D20_old', 'with old model')
print()
print_stats('RBF_QP1D21', 'with RBF')
print_stats('LINEAR_QP1D21', 'with LINEAR')
print_stats('RBF_ghost_QP1D21', 'with RBF')
print_stats('LINEAR_ghost_QP1D21', 'with LINEAR')
print_stats('QP1D21_old', 'with old model')
print()
print_stats('RBF_QP1D22', 'with RBF')
print_stats('LINEAR_QP1D22', 'with LINEAR')
print_stats('RBF_ghost_QP1D22', 'with RBF')
print_stats('LINEAR_ghost_QP1D22', 'with LINEAR')
print_stats('QP1D22_old', 'with old model')
print()
print_stats('RBF_QP1D23', 'with RBF')
print_stats('LINEAR_QP1D23', 'with LINEAR')
print_stats('RBF_ghost_QP1D23', 'with RBF')
print_stats('LINEAR_ghost_QP1D23', 'with LINEAR')
print_stats('QP1D23_old', 'with old model')
print()
print_stats('RBF_QP1D24', 'with RBF')
print_stats('LINEAR_QP1D24', 'with LINEAR')
print_stats('RBF_ghost_QP1D24', 'with RBF')
print_stats('LINEAR_ghost_QP1D24', 'with LINEAR')
print_stats('QP1D24_old', 'with old model')
print()
print_stats('RBF_QP1D25', 'with RBF')
print_stats('LINEAR_QP1D25', 'with LINEAR')
print_stats('RBF_ghost_QP1D25', 'with RBF')
print_stats('LINEAR_ghost_QP1D25', 'with LINEAR')
print_stats('QP1D25_old', 'with old model')
print()
print_stats('RBF_QP1D26', 'with RBF')
print_stats('LINEAR_QP1D26', 'with LINEAR')
print_stats('RBF_ghost_QP1D26', 'with RBF')
print_stats('LINEAR_ghost_QP1D26', 'with LINEAR')
print_stats('QP1D26_old', 'with old model')
print()
print_stats('RBF_QP1D27', 'with RBF')
print_stats('LINEAR_QP1D27', 'with LINEAR')
print_stats('RBF_ghost_QP1D27', 'with RBF')
print_stats('LINEAR_ghost_QP1D27', 'with LINEAR')
print_stats('QP1D27_old', 'with old model')
print()
print_stats('RBF_QP1D29', 'with RBF')
print_stats('LINEAR_QP1D29', 'with LINEAR')
print_stats('RBF_ghost_QP1D29', 'with RBF')
print_stats('LINEAR_ghost_QP1D29', 'with LINEAR')
print_stats('QP1D29_old', 'with old model')
print()
print_stats('RBF_QP1D30', 'with RBF')
print_stats('LINEAR_QP1D30', 'with LINEAR')
print_stats('RBF_ghost_QP1D30', 'with RBF')
print_stats('LINEAR_ghost_QP1D30', 'with LINEAR')
print_stats('QP1D30_old', 'with old model')
print()
print_stats('RBF_QP1D34', 'with RBF')
print_stats('LINEAR_QP1D34', 'with LINEAR')
print_stats('RBF_ghost_QP1D34', 'with RBF')
print_stats('LINEAR_ghost_QP1D34', 'with LINEAR')
print_stats('QP1D34_old', 'with old model')
print()
print_stats('RBF_QP1D35', 'with RBF')
print_stats('LINEAR_QP1D35', 'with LINEAR')
print_stats('RBF_ghost_QP1D35', 'with RBF')
print_stats('LINEAR_ghost_QP1D35', 'with LINEAR')
print_stats('QP1D35_old', 'with old model')
print()
print_stats('RBF_QP1D40', 'with RBF')
print_stats('LINEAR_QP1D40', 'with LINEAR')
print_stats('RBF_ghost_QP1D40', 'with RBF')
print_stats('LINEAR_ghost_QP1D40', 'with LINEAR')
print_stats('QP1D40_old', 'with old model')
print()
print_stats('RBF_QP1D45', 'with RBF')
print_stats('LINEAR_QP1D45', 'with LINEAR')
print_stats('RBF_ghost_QP1D45', 'with RBF')
print_stats('LINEAR_ghost_QP1D45', 'with LINEAR')
print_stats('QP1D45_old', 'with old model')
print()

Average Test Score RBF_QP1D1 with RBF: 0.635
Standard Deviation of Test Score RBF_QP1D1 with RBF: 0.0412
Maximum Test Score RBF_QP1D1 with RBF: 0.7
Minimum Test Score RBF_QP1D1 with RBF: 0.55
------------------------------------------------------
Average Test Score LINEAR_QP1D1 with LINEAR: 0.785
Standard Deviation of Test Score LINEAR_QP1D1 with LINEAR: 0.0747
Maximum Test Score LINEAR_QP1D1 with LINEAR: 0.9
Minimum Test Score LINEAR_QP1D1 with LINEAR: 0.65
------------------------------------------------------
Average Test Score RBF_ghost_QP1D1 with RBF: 0.555
Standard Deviation of Test Score RBF_ghost_QP1D1 with RBF: 0.055
Maximum Test Score RBF_ghost_QP1D1 with RBF: 0.65
Minimum Test Score RBF_ghost_QP1D1 with RBF: 0.5
------------------------------------------------------
Average Test Score LINEAR_ghost_QP1D1 with LINEAR: 0.74
Standard Deviation of Test Score LINEAR_ghost_QP1D1 with LINEAR: 0.0316
Maximum Test Score LINEAR_ghost_QP1D1 with LINEAR: 0.8
Minimum Test Score LINEAR_gho